# ElevenLabs Python Infinite TTS API Google Colab - 11Labs_TTS_Colab

GitHub: [https://github.com/elevenlabs/elevenlabs-python](https://github.com/elevenlabs/elevenlabs-python)

Website: [https://elevenlabs.io/](https://elevenlabs.io/)

# 1. Konfiguracja Wstępna✅

In [ ]:
# @title 1.1 Instalacja Zależności { run: "auto" }

!pip install elevenlabs -U
!pip install rich -U
!pip install pysrt -U
!pip install pydub -U
!pip install nest_asyncio -U

#===============================================================================
# ExecutionTimer - Mierzenie Czasu
"""
    Module execution_timer provides an ExecutionTimer class
        to measure the execution time of a code block.
    It offers two usage options: as a context manager and as a decorator.

    * Example usage as a context manager:
        with ExecutionTimer():
            main()

    * Example usage as a decorator:
        @execution_timer
        def main():
            # Code block to measure execution time
"""

from datetime import datetime
from time import perf_counter_ns
from dataclasses import dataclass
from rich.console import Console


@dataclass(slots=True)
class ExecutionTimer:
    """
        ExecutionTimer is a context manager that measures the execution time of a code block.
        It captures the start time, end time, and duration of the code block.
    """

    start_date: datetime = None
    end_date: datetime = None
    start_time_ns: int = None
    end_time_ns: int = None
    console: Console = Console()

    def __post_init__(self):
        self.start_date = datetime.now()
        self.start_time_ns = perf_counter_ns()

    def __enter__(self) -> 'ExecutionTimer':
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            self.end_date = datetime.now()
            self.end_time_ns = perf_counter_ns()
            self.display_time()
        except AttributeError:
            print('An error occurred: __exit__')

    @staticmethod
    def current_datetime(date: datetime) -> str:
        """
            Formats a datetime object as a string in the format 'YYYY-MM-DD HH:MM:SS'.
        """

        return f'[yellow]{date.year}-{date.month:02d}-{date.day:02d}' \
               f' [white bold]{date.hour:02d}:{date.minute:02d}:{date.second:02d}'

    def calculate_duration(self) -> str:
        """
            Calculates the duration of the code block in hours, minutes, seconds, milliseconds,
            microseconds, and nanoseconds.
        """

        duration_ns: int = self.end_time_ns - self.start_time_ns
        duration_s, duration_ns = map(int, divmod(duration_ns, 1_000_000_000))
        duration_ms, duration_ns = map(int, divmod(duration_ns, 1_000_000))
        duration_us, duration_ns = map(int, divmod(duration_ns, 1_000))

        hours, remainder = map(int, divmod(duration_s, 3600))
        minutes, seconds = map(int, divmod(remainder, 60))

        return f'[white bold]{hours:02d}:{minutes:02d}:{seconds:02d}:' \
               f'{duration_ms:03d}:{duration_us:03d}:{duration_ns:03d}'

    def calculate_duration_alt(self) -> tuple[float, ...]:
        """
            Calculates the duration of the code block in hours, minutes, and seconds
            using an alternative method.
        """

        duration_ns: int = self.end_time_ns - self.start_time_ns
        hours_alt: float = duration_ns / 1_000_000_000 / 60 / 60
        minutes_alt: float = duration_ns / 1_000_000_000 / 60
        seconds_alt: float = duration_ns / 1_000_000_000

        return hours_alt, minutes_alt, seconds_alt

    def display_time(self):
        """
            Displays the start date, end date, and duration of the code block execution.
        """

        start_date_str: str = self.current_datetime(self.start_date)
        end_date_str: str = self.current_datetime(self.end_date)
        duration: str = self.calculate_duration()
        hours_alt, minutes_alt, seconds_alt = map(
            float, self.calculate_duration_alt())

        self.console.print('\n[bold white]╚═══════════ EXECUTION TIME ═══════════╝')
        self.console.print('[bold bright_yellow]        YYYY-MM-DD HH:MM:SS:ms :µs :ns')
        self.console.print(f'[bright_red bold][[bold white]START[bright_red bold]] {start_date_str}')
        self.console.print(f'[bright_red bold][[bold white]END[bright_red bold]]   {end_date_str}')
        self.console.print(f'[bright_red bold][[bold white]TIME[bright_red bold]]  [bold bright_yellow]YYYY-MM-DD {duration}')
        self.console.print('[bright_red bold]                   ^^^^^^^^^^^^')
        self.console.print(f'[bright_red bold][[bold white]TIME[bright_red bold]]  [white bold]{hours_alt:.9f} hours')
        self.console.print(f'[bright_red bold][[bold white]TIME[bright_red bold]]  [white bold]{minutes_alt:.9f} minutes')
        self.console.print(f'[bright_red bold][[bold white]TIME[bright_red bold]]  [white bold]{seconds_alt:.9f} seconds')


def execution_timer(func):
    """
        Decorator that measures the execution time of a function using ExecutionTimer.
    """

    def wrapper(*args, **kwargs):
        with ExecutionTimer():
            result = func(*args, **kwargs)
        return result

    return wrapper

In [ ]:
# @title 1.2.1 Zamontuj Google Drive i Stwórz Katalog: 11Labs_TTS_Colab { run: "auto" }
import os

from google.colab import drive
drive.mount('/content/drive')

# Ścieżka do katalogu, który chcesz utworzyć
google_drive_path = '/content/drive/MyDrive/11Labs_TTS_Colab/'
# google_drive_path = '/content/drive/Shareddrives/Dysk_0/11Labs_TTS_Colab/'

# Tworzenie katalogu, jeśli nie istnieje
if not os.path.exists(google_drive_path):
    os.makedirs(google_drive_path)
    print(f"Utworzono katalog: {google_drive_path}")
else:
    print(f"Katalog już istnieje: {google_drive_path}")

%cd /content/drive/MyDrive/11Labs_TTS_Colab/
print(f"Przejście do: {google_drive_path}")

***lub***

In [ ]:
# @title 1.2.2 Zamontuj Repozytorium GitHub 11Labs_TTS_Colab { run: "auto" }
import os

# Twój Personal Access Token
# https://github.com/settings/tokens
token = 'YOU_token' #@param {type:"string"}

# Adres URL repozytorium
repo_url = 'https://github.com/YOU/11Labs_TTS_Colab.git' #@param {type:"string"}

# Ścieżka do folderu, w którym chcesz umieścić repozytorium
github_repo_path = '/content/11Labs_TTS_Colab/'

# Konstrukcja URL z tokenem autoryzacyjnym
auth_repo_url = repo_url.replace('https://', f'https://{token}@')

# Użycie git clone z autoryzacją
if not os.path.exists(github_repo_path):
    !git clone $auth_repo_url $github_repo_path
    print(f"Utworzono katalog: {github_repo_path}")
else:
    print(f"Katalog już istnieje: {github_repo_path}")


# Wysyłanie zmian do repozytorium
%cd /content/11Labs_TTS_Colab/
print(f"Przejście do: {github_repo_path}")

# Ustawienie konfiguracji dla Git
global_email = "YOU_email"  #@param {type:"string"}
global_name = "YOU_user_name"  #@param {type:"string"}
!git config --global user.email "{global_email}"
!git config --global user.name "{global_name}"
print("Konfiguracja użytkownika Git zakończona!")

# Wysyłanie zmian do repozytorium
# try:
#     print("Wysyłanie zmian do repozytorium:")
#     !git add .
#     !git commit -m "Opis zmian"
#     !git push origin main
#     print("Zmiany zostały wysłane do repozytorium.")
# except Exception as e:
#     print("Wystąpił błąd podczas wysyłania zmian:", e)



# 2. ElevenLabs Infinite TTS API ✅

In [ ]:
# @title 2.1 API Key Manager - Dodaj / Usuń / Zapisz Klucze API Do Google Drive ✅ { run: "auto" }
import os
import json
import ipywidgets as widgets
from IPython.display import display, HTML
import requests
import subprocess

#===============================================================================
# Button CSS
display(HTML('''
<style>
    .widget-button {
        font-weight: bold !important;
    }
    .widget-button.jupyter-widgets.widget-button.mod-active,
    .widget-button:active {
        background-color: #2d2e2e;
    }
    .widget-toggle-button {
        font-weight: bold !important;
    }
    .widget-toggle-button.jupyter-widgets.widget-toggle-button.mod-active,
    .widget-toggle-button:active {
        background-color: #2d2e2e;
    }
</style>
'''))
#===============================================================================

api_keys = []

class APIKeyManager:
    def __init__(self, directory_path, file_name):
        self.directory_path = directory_path
        self.file_name = file_name
        self.file_path = os.path.join(directory_path, file_name)
        self.api_keys = self.load_api_keys()
        self.key_info_output = widgets.Output()

    def load_api_keys(self):
        try:
            with open(self.file_path, 'r') as file:
                return json.load(file)
        except FileNotFoundError:
            return []

    def save_api_keys(self, keys):
        with open(self.file_path, 'w') as file:
            json.dump(keys, file, indent=4)

    def call_api_and_calculate_remaining_characters(self, api_key):
        try:
            url = "https://api.elevenlabs.io/v1/user"
            headers = {
                "Accept": "application/json",
                "xi-api-key": api_key
            }
            response = requests.get(url, headers=headers)
            response_data = response.json()
            character_limit = response_data['subscription']['character_limit']
            character_count = response_data['subscription']['character_count']
            return character_limit - character_count
        except (requests.exceptions.RequestException, KeyError, json.JSONDecodeError) as e:
            print("Wystąpił błąd podczas wywoływania API:", e)
            return None

    def update_and_save_remaining_characters(self, api_key):
        if api_key is None:
            return
        try:
            remaining_characters = self.call_api_and_calculate_remaining_characters(api_key)
            if remaining_characters is not None:
                for key in self.api_keys:
                    if key['key'] == api_key:
                        key['remaining_characters'] = remaining_characters
                        break
                self.save_api_keys(self.api_keys)
        except Exception as e:
            print("Wystąpił błąd podczas aktualizowania pozostałych znaków:", e)

    def generate_new_id(self):
        return max(key['id'] for key in self.api_keys) + 1 if self.api_keys else 1

    def execute_action(self, button):
        selected_action = self.action_radio.value
        if selected_action == 'Dodaj klucz API':
            new_key = self.new_key_text.value
            new_description = self.new_description_text.value
            if new_key and new_description:
                try:
                    if any(key['key'] == new_key or key['description'] == new_description for key in self.api_keys):
                        print("Klucz API lub opis się powtarzają. Proszę wybrać unikalne wartości.")
                    else:
                        new_id = self.generate_new_id()
                        new_api_key = {"id": new_id, "key": new_key, "description": new_description}
                        self.api_keys.append(new_api_key)
                        self.save_api_keys(self.api_keys)
                        print("Nowy klucz API został dodany.")
                        self.update_and_save_remaining_characters(new_key)
                        self.key_info_dropdown.options = [
                        (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in self.api_keys]
                        for option in self.key_info_dropdown.options:
                            if option[1]['id'] == new_id:
                                self.key_info_dropdown.value = option[1]
                                break
                        self.display_key_info(self.key_info_dropdown.value)
                except Exception as e:
                    print("Wystąpił błąd podczas dodawania klucza API:", e)
        elif selected_action == 'Usuń klucz API':
            if self.key_info_dropdown.value is not None:
                try:
                    selected_key = self.key_info_dropdown.value
                    key_id = selected_key['id']
                    for key in self.api_keys:
                        if key['id'] == key_id:
                            self.api_keys.remove(key)
                            self.save_api_keys(self.api_keys)
                            print(f"Klucz API o ID '{key_id}' został usunięty.")
                            self.key_info_dropdown.options = [
                                (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in self.api_keys]
                            self.key_info_dropdown.value = None
                            self.key_info_output.clear_output()
                            break
                except Exception as e:
                    print("Wystąpił błąd podczas usuwania klucza API:", e)
            else:
                print("Wybierz klucz do usunięcia.")

    def display_key_info(self, key):
        self.key_info_output.clear_output()
        with self.key_info_output:
            print(f"ID: {key['id']}")
            print(f"Key: {key['key']}")
            print(f"Description: {key['description']}")
            if 'remaining_characters' in key:
                print(f"Remaining Characters: {key['remaining_characters']}")

    def on_key_info_dropdown_change(self, change):
        if change['new'] is not None:
            self.display_key_info(change['new'])
            self.update_and_save_remaining_characters(change['new']['key'])
            current_key = self.key_info_dropdown.value
            self.key_info_dropdown.options = [
                (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key)
                for key in self.api_keys]
            self.key_info_dropdown.value = current_key
            self.display_key_info(current_key)

    def start(self):
        self.action_radio = widgets.RadioButtons(
            options=['Dodaj klucz API', 'Usuń klucz API'],
            description='Akcja:'
        )

        self.new_key_text = widgets.Text(description='Klucz API:')
        self.new_description_text = widgets.Text(description='Opis:')

        self.key_info_dropdown = widgets.Dropdown(
            options=[(f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key)
                    for key in self.api_keys if 'remaining_characters' in key],
            description='Wybierz:',
            value=self.api_keys[0] if self.api_keys else None
        )

        self.key_info_dropdown.observe(self.on_key_info_dropdown_change, 'value')

        self.key_info_output = widgets.Output()

        self.execute_button = widgets.Button(
            description='Wykonaj',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        self.execute_button.on_click(self.execute_action)

        display(self.action_radio)
        display(self.new_key_text)
        display(self.new_description_text)
        display(self.key_info_dropdown)
        display(self.key_info_output)
        display(self.execute_button)

        if self.api_keys:
            self.display_key_info(self.api_keys[0])
            for key in self.api_keys:
                self.update_and_save_remaining_characters(key['key'])
        self.on_key_info_dropdown_change({'new': self.key_info_dropdown.value})

    def get_api_keys(self):
        global api_keys
        api_keys = self.api_keys

def main(directory_path):
    file_name = 'elevenlabs_api_keys.json'

    display(HTML('<h1>API Key Manager - Google Drive</h1>'))
    manager = APIKeyManager(directory_path, file_name)
    manager.start()
    manager.get_api_keys()

if __name__ == "__main__":
    main(google_drive_path)

***lub***

In [ ]:
# @title 2.2 API Key Manager - Dodaj / Usuń / Zapisz Klucze API Oraz Pliki Audio Do Repozytorium GitHub ✅ { run: "auto" }
import os
import json
import ipywidgets as widgets
from IPython.display import display, HTML
import requests
import subprocess

#===============================================================================
# Button CSS
display(HTML('''
<style>
    .widget-button {
        font-weight: bold !important;
    }
    .widget-button.jupyter-widgets.widget-button.mod-active,
    .widget-button:active {
        background-color: #2d2e2e;
    }
    .widget-toggle-button {
        font-weight: bold !important;
    }
    .widget-toggle-button.jupyter-widgets.widget-toggle-button.mod-active,
    .widget-toggle-button:active {
        background-color: #2d2e2e;
    }
</style>
'''))
#===============================================================================

api_keys = []

class APIKeyManager:
    def __init__(self, token, repo_url, directory_path, auth_repo_url, file_name):
        self.token = token
        self.repo_url = repo_url
        self.directory_path = directory_path
        self.auth_repo_url = auth_repo_url
        self.file_name = file_name
        self.file_path = os.path.join(directory_path, file_name)
        self.api_keys = self.load_api_keys()
        self.key_info_output = widgets.Output()

    def update_github_repo(self, commit_msg):
        try:
            os.chdir(self.directory_path)
            subprocess.run(["git", "add", "."])
            subprocess.run(["git", "commit", "-m", commit_msg])
            subprocess.run(["git", "push", "origin", "main"])
            print("Repozytorium zostało zaktualizowane na GitHubie.")
        except Exception as e:
            print("Wystąpił błąd podczas aktualizowania repozytorium:", e)

    def load_api_keys(self):
        try:
            with open(self.file_path, 'r') as file:
                return json.load(file)
        except FileNotFoundError:
            return []

    def save_api_keys(self, keys):
        with open(self.file_path, 'w') as file:
            json.dump(keys, file, indent=4)

    def call_api_and_calculate_remaining_characters(self, api_key):
        try:
            url = "https://api.elevenlabs.io/v1/user"
            headers = {
                "Accept": "application/json",
                "xi-api-key": api_key
            }
            response = requests.get(url, headers=headers)
            response_data = response.json()
            character_limit = response_data['subscription']['character_limit']
            character_count = response_data['subscription']['character_count']
            return character_limit - character_count
        except (requests.exceptions.RequestException, KeyError, json.JSONDecodeError) as e:
            print("Wystąpił błąd podczas wywoływania API:", e)
            return None

    def update_and_save_remaining_characters(self, api_key):
        if api_key is None:
            return
        try:
            remaining_characters = self.call_api_and_calculate_remaining_characters(api_key)
            if remaining_characters is not None:
                for key in self.api_keys:
                    if key['key'] == api_key:
                        key['remaining_characters'] = remaining_characters
                        break
                self.save_api_keys(self.api_keys)
        except Exception as e:
            print("Wystąpił błąd podczas aktualizowania pozostałych znaków:", e)

    def generate_new_id(self):
        return max(key['id'] for key in self.api_keys) + 1 if self.api_keys else 1

    def execute_action(self, button):
        selected_action = self.action_radio.value
        if selected_action == 'Dodaj klucz API':
            new_key = self.new_key_text.value
            new_description = self.new_description_text.value
            if new_key and new_description:
                try:
                    if any(key['key'] == new_key or key['description'] == new_description for key in self.api_keys):
                        print("Klucz API lub opis się powtarzają. Proszę wybrać unikalne wartości.")
                    else:
                        new_id = self.generate_new_id()
                        new_api_key = {"id": new_id, "key": new_key, "description": new_description}
                        self.api_keys.append(new_api_key)
                        self.save_api_keys(self.api_keys)
                        print("Nowy klucz API został dodany.")
                        self.update_and_save_remaining_characters(new_key)
                        self.key_info_dropdown.options = [
                            (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in self.api_keys]
                        for option in self.key_info_dropdown.options:
                            if option[1]['id'] == new_id:
                                self.key_info_dropdown.value = option[1]
                                break
                        self.display_key_info(self.key_info_dropdown.value)
                    self.update_github_repo("Dodano klucz API")
                except Exception as e:
                    print("Wystąpił błąd podczas dodawania klucza API:", e)
        elif selected_action == 'Usuń klucz API':
            if self.key_info_dropdown.value is not None:
                try:
                    selected_key = self.key_info_dropdown.value
                    key_id = selected_key['id']
                    for key in self.api_keys:
                        if key['id'] == key_id:
                            self.api_keys.remove(key)
                            self.save_api_keys(self.api_keys)
                            print(f"Klucz API o ID '{key_id}' został usunięty.")
                            self.key_info_dropdown.options = [
                                (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in self.api_keys]
                            self.key_info_dropdown.value = None
                            self.key_info_output.clear_output()
                            break
                    self.update_github_repo("Usunięto klucz API")
                except Exception as e:
                    print("Wystąpił błąd podczas usuwania klucza API:", e)
            else:
                print("Wybierz klucz do usunięcia.")


    def display_key_info(self, key):
        self.key_info_output.clear_output()
        with self.key_info_output:
            print(f"ID: {key['id']}")
            print(f"Key: {key['key']}")
            print(f"Description: {key['description']}")
            if 'remaining_characters' in key:
                print(f"Remaining Characters: {key['remaining_characters']}")

    def on_key_info_dropdown_change(self, change):
        if change['new'] is not None:
            self.display_key_info(change['new'])
            self.update_and_save_remaining_characters(change['new']['key'])
            self.update_github_repo("Zaktualizowano dane klucza API")
            current_key = self.key_info_dropdown.value
            self.key_info_dropdown.options = [
                (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key)
                for key in self.api_keys]
            self.key_info_dropdown.value = current_key
            self.display_key_info(current_key)

    def start(self):
        self.action_radio = widgets.RadioButtons(
            options=['Dodaj klucz API', 'Usuń klucz API'],
            description='Akcja:'
        )

        self.new_key_text = widgets.Text(description='Klucz API:')
        self.new_description_text = widgets.Text(description='Opis:')

        self.key_info_dropdown = widgets.Dropdown(
            options=[(f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key)
                    for key in self.api_keys if 'remaining_characters' in key],
            description='Wybierz:',
            value=self.api_keys[0] if self.api_keys else None
        )

        self.key_info_dropdown.observe(self.on_key_info_dropdown_change, 'value')

        self.key_info_output = widgets.Output()

        self.execute_button = widgets.Button(
            description='Wykonaj',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        self.execute_button.on_click(self.execute_action)

        display(self.action_radio)
        display(self.new_key_text)
        display(self.new_description_text)
        display(self.key_info_dropdown)
        display(self.key_info_output)
        display(self.execute_button)

        if self.api_keys:
            self.display_key_info(self.api_keys[0])
            for key in self.api_keys:
                self.update_and_save_remaining_characters(key['key'])

        self.on_key_info_dropdown_change({'new': self.key_info_dropdown.value})
        self.update_github_repo("Automatyczna aktualizacja pozostałych znaków")

    def get_api_keys(self):
        global api_keys
        api_keys = self.api_keys

def main(token, repo_url, github_repo_path, auth_repo_url):
    directory_path = github_repo_path
    file_name = 'elevenlabs_api_keys.json'

    display(HTML('<h1>API Key Manager - Repozytorium GitHub</h1>'))
    manager = APIKeyManager(token, repo_url, directory_path, auth_repo_url, file_name)
    manager.start()
    manager.get_api_keys()

if __name__ == "__main__":
    main(token, repo_url, github_repo_path, auth_repo_url)

In [ ]:
# @title ElevenLabs Infinite TTS API Generator { run: "auto" }

import ipywidgets as widgets
import IPython.display as ipd
import requests
import os
import zipfile
from elevenlabs import voices, set_api_key, Models
from IPython.display import clear_output, display, HTML
from google.colab import files


class TTSGenerator:
    def __init__(self):
        self.output_dir = 'output_audio'
        self.default_path = "/content/"
        self.google_drive_path = '/content/drive/MyDrive/11Labs_TTS_Colab/'
        self.github_repo_path = '/content/11Labs_TTS_Colab/'
        self.available_paths = self.get_available_paths()
        self.folder_radio = self.create_folder_radio()
        self.folder_radio.observe(self.update_folder, 'value')
        self.api_key_widget = self.create_api_key_widget()
        self.api_key_widget.observe(self.set_api_and_refresh, names='value')
        self.audio_files = []
        self.url = None
        self.headers = None

        os.chdir(self.default_path)

    def get_available_paths(self):
        available_paths = []
        if os.path.exists(self.default_path):
            available_paths.append('Google Colab')
        if self.google_drive_path is not None:
            available_paths.append('Google Drive')
        if self.github_repo_path is not None:
            available_paths.append('GitHub')
        return available_paths

    def create_folder_radio(self):
        return widgets.RadioButtons(
            options=self.available_paths,
            description='Lokalizacja:',
            disabled=False
        )

    def update_folder(self, change):
        new_path = self.default_path
        if change.new == 'Google Drive' and self.google_drive_path is not None:
            new_path = self.google_drive_path
        if change.new == 'GitHub' and self.github_repo_path is not None:
            new_path = self.github_repo_path
        os.chdir(new_path)
        os.makedirs(self.output_dir, exist_ok=True)
        self.audio_files = self.refresh_output_audio_list(self.output_dir)
        self.selected_audio.options = self.audio_files

    def create_api_key_widget(self):
        sorted_api_keys = sorted(api_keys, key=lambda k: k['id'], reverse=False)
        first_valid_key = next((key for key in sorted_api_keys if key.get('remaining_characters', 0) > 0), None)
        return widgets.Dropdown(
            options=[(f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key['key']) for key in sorted_api_keys],
            description="Klucz API:",
            value=first_valid_key['key'] if first_valid_key else None
        )

    def set_api_and_refresh(self, change):
        try:
            selected_key = next(
                key for key in api_keys if key['key'] == self.api_key_widget.value)
            set_api_key(selected_key['key'])
            self.refresh_voices_and_clear_ui()
        except Exception as e:
            clear_output()
            display(HTML('<h1>ElevenLabs TTS API Google Colab</h1>'))
            display(self.api_key_widget)
            display(HTML('<p style="color: red;">Błąd: Nieprawidłowy klucz API</p>'))

    def display_api_generator_header(self):
        clear_output()
        display(HTML('<h1>ElevenLabs Infinite TTS API Generator</h1>'))
        set_api_key(self.api_key_widget.value)

    def get_voice_options(self):
        available_voices = voices()
        return sorted([
            (
                f"{voice.name} ({voice.category}) - {voice.labels['accent']}" + (f", {voice.labels['description']}" if 'description' in voice.labels else "") +
                f", {voice.labels['age']}, {voice.labels['gender']}" +
                (f", {voice.labels['use case']}" if 'use case' in voice.labels else ""),
                voice.voice_id
            )
            for voice in available_voices.voices
        ])

    def create_selected_voice_dropdown(self, voice_options):
        return widgets.Dropdown(
            options=voice_options,
            description="Głos:"
        )

    def display_selected_voice_preview(self, change):
        selected_voice_id = change['new']  # Dodaj tę linijkę
        with self.output_audio_widget:
            self.output_audio_widget.clear_output()
            selected_voice_id = change['new']
            selected_voice = next((voice for voice in voices(
            ).voices if voice.voice_id == selected_voice_id), None)
            if selected_voice:
                print(f"Podgląd Głosu: {selected_voice.name}, VID: {selected_voice_id}")
                # Automatyczne odtwarzanie
                # audio_display = ipd.Audio(
                #     selected_voice.preview_url, autoplay=True)
                audio_display = ipd.Audio(
                    selected_voice.preview_url, autoplay=False)
                display(audio_display)
                if selected_voice.settings:
                    self.stability_widget.value = selected_voice.settings.stability
                    self.similarity_boost_widget.value = selected_voice.settings.similarity_boost

    def setup_voice_change_handler(self, selected_voice):
        selected_voice.observe(
            self.display_selected_voice_preview, names='value')
        selected_voice.observe(lambda change: self.update_url_and_headers(
            selected_voice), names='value')

    def initialize_voice_settings(self, selected_voice):
        self.update_url_and_headers(selected_voice)

    def setup_selected_voice_dropdown(self):
        voice_options = self.get_voice_options()
        selected_voice = self.create_selected_voice_dropdown(voice_options)
        self.setup_voice_change_handler(selected_voice)
        self.initialize_voice_settings(selected_voice)
        return selected_voice

    def create_text_widget(self):
        text_widget = widgets.Textarea(
            value="OK",
            description="Tekst:",
            rows=5,
            layout=widgets.Layout(width="90%", height="150px")
        )

        char_counter = widgets.Label(
            value=f"{len(text_widget.value)} / 2500",
            layout=widgets.Layout(margin='0 0 0 90px')  # Dodaj tę linijkę
        )

        def check_length(change):
            if len(change.new) > 2500:
                text_widget.value = change.new[:2500]
            char_counter.value = f"{len(text_widget.value)} / 2500"

        text_widget.observe(check_length, 'value')

        return widgets.VBox([text_widget, char_counter])

    def update_url_and_headers(self, selected_voice):
        self.url = f"https://api.elevenlabs.io/v1/text-to-speech/{selected_voice.value}"
        self.headers = {
            "Accept": "audio/mpeg",
            "Content-Type": "application/json",
            "xi-api-key": self.api_key_widget.value
        }

    def create_model_name_to_id_mapping(self, models_list):
        return {model.name: model.model_id for model in models_list.models}

    def create_model_name_widget(self, models_list):
        return widgets.Dropdown(
            options=[model.name for model in models_list.models],
            value=models_list.models[0].name,
            description="Model:"
        )

    def create_model_widgets(self):
        models_list = Models.from_api()
        model_name_to_id = self.create_model_name_to_id_mapping(models_list)
        model_name_widget = self.create_model_name_widget(models_list)
        return model_name_to_id, model_name_widget

    def create_stability_widget(self):
        return widgets.FloatSlider(
            value=0.5,
            min=0.0,
            max=1.0,
            step=0.01,
            description="Stabilność:"
        )

    def create_similarity_boost_widget(self):
        return widgets.FloatSlider(
            value=0.75,
            min=0.0,
            max=1.0,
            step=0.01,
            description="Podobność:"
        )

    def create_style_widget(self):
        return widgets.FloatSlider(
            value=0.0,
            min=0.0,
            max=1.0,
            step=0.01,
            description="Styl:"
        )

    def create_use_speaker_boost_widget(self):
        return widgets.RadioButtons(
            options=[("Tak", True), ("Nie", False)],
            value=True,
            description="Przesilenie:"
        )

    def create_unique_filename(self, output_dir, counter):
        while True:
            filename = os.path.join(output_dir, f"{counter:05d}.mp3")
            if not os.path.exists(filename):
                return filename
            counter += 1

    def refresh_output_audio_list(self, output_dir):
        return sorted(
            [
                filename
                for filename in os.listdir(output_dir)
                if filename.endswith('.mp3')
            ]
        )

    def download_audio(self, b, text_widget, model_name_widget, model_name_to_id, stability_widget,
                      similarity_boost_widget, style_widget, use_speaker_boost_widget,
                      selected_voice, selected_audio, url, headers, output_dir):

        self.audio_files = self.refresh_output_audio_list(output_dir)

        data = {
            "text": text_widget.value,
            "model_id": model_name_to_id[model_name_widget.value],
            "voice_settings": {
                "stability": stability_widget.value,
                "similarity_boost": similarity_boost_widget.value,
                "style": style_widget.value,
                "use_speaker_boost": use_speaker_boost_widget.value
            }
        }

        response = requests.post(url, json=data, headers=headers)

        output_filename = self.create_unique_filename(
            output_dir, len(self.audio_files) + 1)

        with open(output_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=2048):
                if chunk:
                    f.write(chunk)

        self.audio_files.append(os.path.basename(output_filename))
        selected_audio.options = self.audio_files
        selected_audio.value = [os.path.basename(output_filename)]

    def display_selected_audio(self, change, output_dir, selected_audio, output_audio_widget):
        with output_audio_widget:
            output_audio_widget.clear_output()
            selected_files = selected_audio.value
            if len(selected_files) == 1:
                filename = selected_files[0]
                audio_path = os.path.join(output_dir, filename)
                display(HTML(f'<h3>Plik: {filename}</h3>'))
                # Automatyczne odtwarzanie
                audio_display = ipd.Audio(audio_path, autoplay=True)
                display(audio_display)
            else:
                for filename in selected_files:
                    audio_path = os.path.join(output_dir, filename)
                    display(HTML(f'<h3>Plik: {filename}</h3>'))
                    audio_display = ipd.Audio(audio_path)
                    display(audio_display)

    def download_selected_audio(self, b, output_dir, selected_audio):
        selected_files = selected_audio.value
        zip_filename = os.path.join(output_dir, 'selected_audio.zip')
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for filename in selected_files:
                audio_path = os.path.join(output_dir, filename)
                zipf.write(audio_path, os.path.basename(audio_path))
        files.download(zip_filename)

    def download_all_audio(self, b, output_dir, audio_files):
        zip_filename = os.path.join(output_dir, 'all_audio.zip')
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for filename in audio_files:
                audio_path = os.path.join(output_dir, filename)
                zipf.write(audio_path, os.path.basename(audio_path))
        files.download(zip_filename)

    def create_generating_button(self):
        return widgets.Button(
            description="Wygeneruj Audio",
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width="300px", height="30px"))

    def create_selected_audio(self):
        audio_files = sorted([filename for filename in os.listdir(
            'output_audio') if filename.endswith('.mp3')])
        return widgets.SelectMultiple(
            options=audio_files,
            description='',
            disabled=False
        )

    def create_download_buttons(self, output_dir, selected_audio, audio_files):
        download_selected_button = widgets.Button(
            description='Pobierz Zaznaczone',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px'))

        download_all_button = widgets.Button(
            description='Pobierz Wszystko',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px'))

        download_selected_button.on_click(
            lambda b: self.download_selected_audio(b, output_dir, selected_audio))
        download_all_button.on_click(
            lambda b: self.download_all_audio(b, output_dir, audio_files))

        return download_selected_button, download_all_button

    def refresh_voices_and_clear_ui(self):
        os.makedirs(self.output_dir, exist_ok=True)
        self.folder_radio.observe(self.update_folder, 'value')
        self.display_api_generator_header()
        self.selected_voice = self.setup_selected_voice_dropdown()

        self.text_widget = self.create_text_widget()
        self.model_name_to_id, self.model_name_widget = self.create_model_widgets()

        self.stability_widget = self.create_stability_widget()
        self.similarity_boost_widget = self.create_similarity_boost_widget()
        self.style_widget = self.create_style_widget()
        self.use_speaker_boost_widget = self.create_use_speaker_boost_widget()

        self.selected_audio = self.create_selected_audio()

        if os.path.exists(self.output_dir):
            self.audio_files = self.refresh_output_audio_list(self.output_dir)
            self.selected_audio.options = self.audio_files

        self.audio_files = self.refresh_output_audio_list(self.output_dir)

        self.generating_button = self.create_generating_button()

        self.output_audio_widget = widgets.Output()
        self.display_selected_voice_preview({'new': self.selected_voice.value})

        self.selected_audio = self.create_selected_audio()

        # Download buttons
        self.download_selected_button, self.download_all_button = self.create_download_buttons(
            self.output_dir, self.selected_audio, self.audio_files)

        self.generating_button.on_click(
            lambda b: self.download_audio(
                b, self.text_widget.children[0], self.model_name_widget, self.model_name_to_id, self.stability_widget,
                self.similarity_boost_widget, self.style_widget, self.use_speaker_boost_widget,
                self.selected_voice, self.selected_audio, self.url, self.headers, self.output_dir
            )
        )

        self.selected_audio.observe(
            lambda change: self.display_selected_audio(
                change, self.output_dir, self.selected_audio, self.output_audio_widget),
            names='value'
        )

        #===============================================================================
        # Button CSS
        display(HTML('''
        <style>
            .widget-button {
                font-weight: bold !important;
            }
            .widget-button.jupyter-widgets.widget-button.mod-active,
            .widget-button:active {
                background-color: #2d2e2e;
            }
            .widget-toggle-button {
                font-weight: bold !important;
            }
            .widget-toggle-button.jupyter-widgets.widget-toggle-button.mod-active,
            .widget-toggle-button:active {
                background-color: #2d2e2e;
            }
        </style>
        '''))
        #===============================================================================

        display(self.folder_radio,
                self.api_key_widget,
                self.selected_voice,
                self.model_name_widget,
                self.text_widget,
                self.stability_widget,
                self.similarity_boost_widget,
                self.style_widget,
                self.use_speaker_boost_widget,
                self.generating_button,
                self.selected_audio,
                self.output_audio_widget,
                self.download_selected_button,
                self.download_all_button)

def main():
    tts_generator = TTSGenerator()
    display(tts_generator.api_key_widget)
    tts_generator.refresh_voices_and_clear_ui()

if __name__ == "__main__":
    main()


# 3. ElevenLabs Infinite TTS API - Anime Lektor PL ✅

In [ ]:
# @title ElevenLabs Infinite TTS API - Anime Lektor PL { run: "auto" }
import os
import shutil
import glob
import asyncio
import nest_asyncio
import aiohttp
import pysrt
import zipfile
from pydub import AudioSegment
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import files
from elevenlabs import set_api_key, voices

#===============================================================================
# Button CSS
display(HTML('''
<style>
    .widget-button {
        font-weight: bold !important;
    }
    .widget-button.jupyter-widgets.widget-button.mod-active,
    .widget-button:active {
        background-color: #2d2e2e;
    }
    .widget-toggle-button {
        font-weight: bold !important;
    }
    .widget-toggle-button.jupyter-widgets.widget-toggle-button.mod-active,
    .widget-toggle-button:active {
        background-color: #2d2e2e;
    }
</style>
'''))
#===============================================================================

os.chdir('/content/')
os.chdir('/content/drive/Shareddrives/Dysk_0/11Labs_TTS_Colab/')
nest_asyncio.apply()

class AudioGenerator:
    def __init__(self, input_folder='input', output_folder='audio_output'):
        self.api_keys = []
        self.api_key_statuses = {key['key']: True for key in self.api_keys}
        self.voice_ids = {key['key']: None for key in self.api_keys}
        self.premade_voice_id = None

        self.auto_download = False

        self.audio_cache = {}
        self.use_cache = False

        self.input_folder = input_folder
        self.output_folder = output_folder

        os.makedirs(input_folder, exist_ok=True)
        os.makedirs(output_folder, exist_ok=True)

        self.voice_names = {
            'Bella - Normalna Kobieta': ['Bella'],
            'Brian - Powolny Narrator Dokumentalny': ['Brian', 'Brian - Slow speaking and deep'],
            'Knightley - Głęboki Narrator Książkowy': ['Knightley', 'Knightley - dapper and deep narrator'],
            'Joanne - Pozytywna Introspekcyjna Kobieta': ['Joanne', 'Joanne - pensive, introspective'],
            'Scarlett - Uwodzicielska Niestabilna Kobieta': ['Scarlett', 'Scarlett - 💯% seductive 🔥'],
            'Nicole - Szepcząca Kobieta': ['Nicole'],
            'Wanda - Spokojna Kobieta': ['Wanda', 'Wanda - calm female'],
            'Hallie - Miękko Mówiąca Subtelna Kobieta': ['Hallie', 'Hallie - soft-spoken and subtle'],
        }

        self.voice_dropdown = widgets.Dropdown(options=list(self.voice_names.keys()))
        display(self.voice_dropdown)

    def update_api_keys(self, api_keys):
        self.api_keys = api_keys
        self.api_key_statuses = {key['key']: True for key in api_keys}
        self.voice_ids = {key['key']: None for key in api_keys}

    def create_headers(self, api_key):
        return {
            "Accept": "audio/mpeg",
            "Content-Type": "application/json",
            "xi-api-key": api_key
        }

    def create_data(self, text):
        return {
            "text": text,
            "model_id": "eleven_multilingual_v2",
            "voice_settings": {
                "stability": 1,
                "similarity_boost": 0.75
            }
        }

    def find_voice_id(self, voices, voice_names):
        for voice in voices.voices:
            if voice.name in voice_names:
                if voice.category == 'premade':
                    self.premade_voice_id = voice.voice_id
                return voice.voice_id
        return None

    async def generate_audio_url(self, api_key):
        set_api_key(api_key)
        if self.premade_voice_id is not None:
            return f"https://api.elevenlabs.io/v1/text-to-speech/{self.premade_voice_id}"
        elif self.voice_ids[api_key] is None:
            voice_list = voices()
            self.voice_ids[api_key] = self.find_voice_id(voice_list, self.voice_names[self.voice_dropdown.value])
        if self.voice_ids[api_key]:
            return f"https://api.elevenlabs.io/v1/text-to-speech/{self.voice_ids[api_key]}"
        else:
            return None

    async def write_audio_file(self, response, audio_file_path, CHUNK_SIZE):
        with open(audio_file_path, 'wb') as f:
            while True:
                chunk = await response.content.read(CHUNK_SIZE)
                if not chunk:
                    break
                f.write(chunk)

    async def generate_audio(self, text, index, api_key, temp_folder):
        CHUNK_SIZE = 1024
        url = await self.generate_audio_url(api_key)
        if url is None:
            print(f"Klucz API {api_key} nie ma dostępu do wybranego głosu.")
            self.api_key_statuses[api_key] = False
            return None, False
        headers = self.create_headers(api_key)
        data = self.create_data(text)

        audio_file_path = f'{temp_folder}/{index}.mp3'

        if self.use_cache and text in self.audio_cache:
            return self.audio_cache[text], True

        async with aiohttp.ClientSession() as session:
            async with session.post(url, json=data, headers=headers) as response:
                if response.status == 200:
                    await self.write_audio_file(response, audio_file_path, CHUNK_SIZE)
                    self.audio_cache[text] = audio_file_path
                    return audio_file_path, False
                else:
                    self.api_key_statuses[api_key] = False
                    return None, False

    def copy_audio_file(self, audio_file_path, temp_folder, index):
        new_audio_file_path = f'{temp_folder}/{index}.mp3'
        if audio_file_path != new_audio_file_path:
            shutil.copy(audio_file_path, new_audio_file_path)
        return new_audio_file_path

    def print_subtitle_info(self, index, text, char_count, from_cache):
        print(f"{index}. {text} RC: {char_count if not from_cache else 0}")

    def load_srt_file(self, filename):
        return pysrt.open(filename, encoding='utf-8')

    # def convert_audio_files(self, subs, temp_folder):
    #     for index, _ in enumerate(subs, start=1):
    #         audio_file = f'{temp_folder}/{index}.mp3'
    #         audio = AudioSegment.from_mp3(audio_file)
    #         audio.export(audio_file.replace('.mp3', '.wav'), format='wav')

    def convert_audio_files(self, subs, temp_folder):
        for sub in subs:
            audio_file = f'{temp_folder}/{sub.index}.mp3'
            if os.path.exists(audio_file):
                audio = AudioSegment.from_mp3(audio_file)
                audio.export(audio_file.replace('.mp3', '.wav'), format='wav')
            else:
                print(f"Plik {audio_file} nie istnieje, pomijam.")

    # def combine_audio(self, subs, filename, temp_folder):
    #     combined = AudioSegment.silent(duration=0)

    #     for index, sub in enumerate(subs, start=1):
    #         start_time = sub.start.ordinal
    #         silence = AudioSegment.silent(duration=start_time - len(combined))
    #         combined += silence

    #         audio_file = f'{temp_folder}/{index}.wav'
    #         audio = AudioSegment.from_wav(audio_file)

    #         combined += audio

    #     output_filename = os.path.basename(filename).replace('.srt', '.eac3')
    #     combined.export(
    #         f"{self.output_folder}/{output_filename}", format='wav')
    #     os.system(
    #         f"ffmpeg -i {self.output_folder}/{output_filename} -c:a {self.output_folder}/{output_filename}")
    def combine_audio(self, subs, filename, temp_folder):
        combined = AudioSegment.silent(duration=0)

        for sub in subs:
            start_time = sub.start.ordinal
            silence = AudioSegment.silent(duration=start_time - len(combined))
            combined += silence

            audio_file = f'{temp_folder}/{sub.index}.wav'
            if os.path.exists(audio_file):
                audio = AudioSegment.from_wav(audio_file)
                combined += audio
            else:
                print(f"Plik {audio_file} nie istnieje, pomijam.")

        output_filename = os.path.basename(filename).replace('.srt', '.eac3')
        combined.export(
            f"{self.output_folder}/{output_filename}", format='wav')
        os.system(
            f"ffmpeg -i {self.output_folder}/{output_filename} -c:a {self.output_folder}/{output_filename}")

    async def process_file(self, filename):
        print(f"Przetwarzanie pliku: {filename}")
        base_filename = os.path.basename(filename)
        temp_folder = self.create_temp_folder(base_filename)

        subs = self.load_srt_file(filename)
        await self.process_subtitles(subs, temp_folder)
        self.convert_audio_files(subs, temp_folder)
        self.combine_audio(subs, filename, temp_folder)
        print(f"Przetworzono plik: {filename}")

    # def create_temp_folder(self, base_filename):
    #     temp_folder = f'tmp_output_{base_filename}'
    #     os.makedirs(temp_folder, exist_ok=True)
    #     return temp_folder
    def create_temp_folder(self, base_filename):
        base_filename_without_ext = os.path.splitext(base_filename)[0]
        temp_folder = f'tmp_output_{base_filename_without_ext}'
        os.makedirs(temp_folder, exist_ok=True)
        return temp_folder

    # async def process_subtitles(self, subs, temp_folder):
    #     total_char_count = 0
    #     for index, sub in enumerate(subs, start=1):
    #         char_count = len(sub.text)
    #         for api_key_dict in self.api_keys:
    #             api_key = api_key_dict['key']
    #             if self.api_key_statuses[api_key]:
    #                 audio_file_path, from_cache = await self.generate_audio(sub.text, index, api_key, temp_folder)
    #                 if audio_file_path:
    #                     new_audio_file_path = self.copy_audio_file(audio_file_path, temp_folder, index)
    #                     self.print_subtitle_info(index, sub.text, char_count, from_cache)
    #                     total_char_count += char_count if not from_cache else 0
    #                     break
    #         else:
    #             print("Wszystkie klucze API są wyczerpane. Proszę dodać więcej kluczy.")
    #     print(f"Łączna liczba znaków: {total_char_count}")
    async def process_subtitles(self, subs, temp_folder):
        total_char_count = 0
        for sub in subs:
            char_count = len(sub.text)
            audio_file_path = f'{temp_folder}/{sub.index}.mp3'
            if os.path.exists(audio_file_path):
                print(f"Plik {audio_file_path} już istnieje, pomijam.")
                continue
            for api_key_dict in self.api_keys:
                api_key = api_key_dict['key']
                if self.api_key_statuses[api_key]:
                    audio_file_path, from_cache = await self.generate_audio(sub.text, sub.index, api_key, temp_folder)
                    if audio_file_path:
                        new_audio_file_path = self.copy_audio_file(audio_file_path, temp_folder, sub.index)
                        self.print_subtitle_info(sub.index, sub.text, char_count, from_cache)
                        total_char_count += char_count if not from_cache else 0
                        break
            else:
                print("Wszystkie klucze API są wyczerpane. Proszę dodać więcej kluczy.")
        print(f"Łączna liczba znaków: {total_char_count}")

    async def process_all_files(self):
        files = glob.glob(f'{self.input_folder}/*.srt')
        files.sort()
        for filename in files:
            await self.process_file(filename)

    def start_processing(self):
        upload_button = self.create_upload_button()
        display(upload_button)

        cache_button = self.create_cache_button()
        display(cache_button)

        auto_download_button = self.create_auto_download_button()
        display(auto_download_button)

        start_button = self.create_start_button()
        display(start_button)

    def upload_srt_files(self, b=None):
        try:
            uploaded = files.upload()

            for fn in uploaded.keys():
                print('User uploaded file "{name}" with length {length} bytes'.format(
                    name=fn, length=len(uploaded[fn])))

            for fn in uploaded.keys():
                shutil.move(fn, f'{self.input_folder}/{fn}')
        except:
            pass

    def create_upload_button(self):
        upload_button = widgets.Button(
            description='Prześlij Pliki .SRT',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        upload_button.on_click(self.upload_srt_files)
        return upload_button

    def create_cache_button(self):
        cache_button = widgets.ToggleButton(
            value=self.use_cache,
            description='Użyj Cache - Zaoszczędź RC',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        cache_button.observe(self.update_cache_status, names='value')
        return cache_button

    def update_cache_status(self, change):
        self.use_cache = change['new']

    def create_auto_download_button(self):
        auto_download_button = widgets.ToggleButton(
            value=False,
            description='Automatyczne Pobieranie',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        auto_download_button.observe(self.update_auto_download_status, names='value')
        return auto_download_button

    def update_auto_download_status(self, change):
        self.auto_download = change['new']

    def create_download_button(self):
        download_button = widgets.Button(
            description='Pobierz Pliki',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        download_button.on_click(self.download_files)
        return download_button

    def download_files(self, b=None):
        zip_filename = 'audio_files.zip'
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for filename in glob.glob(f'{self.output_folder}/*.eac3'):
                zipf.write(filename)
        files.download(zip_filename)


    def create_start_button(self):
        start_button = widgets.Button(
            description='Wygeneruj Audio',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        start_button.on_click(self.start_processing_callback)
        return start_button

    @execution_timer
    def start_processing_callback(self, b):
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.process_all_files())
        if self.auto_download:
            self.download_files()
        download_button = self.create_download_button()
        display(download_button)

def main():
    display(HTML('<h1>ElevenLabs Infinite TTS API - Anime Lektor PL</h1>'))
    audio_generator = AudioGenerator()
    audio_generator.update_api_keys(api_keys)
    audio_generator.start_processing()

if __name__ == "__main__":
    main()

In [ ]:
<break time="1.5s" />

# INNE PRZYDATNE ❌

## Przykład z Biblioteką Pythona Elevenlabs

In [ ]:
# from elevenlabs import voices, generate
# from elevenlabs import generate, play, set_api_key
# import rich

# audio = generate(
#     text="Witaj świecie, jak się masz?",
#     api_key="69629e2241ca3f96a3e97587ad5072e5",
#     voice="Scarlett - 💯% seductive 🔥",
#     model='eleven_multilingual_v2'
# )
# # play(audio)
# play(audio, notebook=True)  # notebook=True - odtwarzanie w google colab

# f = open("output.mp3", "wb")
# f.write(audio)
# f.close()


import requests

CHUNK_SIZE = 1024
url = "https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL"

headers = {
  "Accept": "audio/mpeg",
  "Content-Type": "application/json",
  "xi-api-key": "4d88c7168ddf8a163ffbf176276e72c5"
}

data = {
  "text": "Witaj świecie, jak się masz?",
  "model_id": "eleven_multilingual_v2",
  "voice_settings": {
    "stability": 0.5,
    "similarity_boost": 0.75
  }
}

response = requests.post(url, json=data, headers=headers)
with open('output.mp3', 'wb') as f:
    for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

## Lisa Głosów i Ich Ustawień / Wyszukiwanie głosów po ich nazwach



In [ ]:
# pip install elevenlabs
# pip install rich
from elevenlabs import voices
from elevenlabs import set_api_key
import rich

# Lisa głosów i ich ustawień

# set_api_key("69629e2241ca3f96a3e97587ad5072e5")
# set_api_key("96028d102d0734fd815519ead6c6a625")
set_api_key("77c2a2122bbfeb8e570f91ef3cb27982")

voices = voices()
rich.print(voices)


# import requests

# url = "https://api.elevenlabs.io/v1/voices/"

# headers = {
#     "Accept": "application/json",
#     "xi-api-key": "69629e2241ca3f96a3e97587ad5072e5"
# }

# response = requests.get(url, headers=headers)

# rich.print(response.text)

In [ ]:
## Wyszukiwanie id głosu po jego nazwach
from elevenlabs import voices
from elevenlabs import set_api_key
import rich

def find_voice_id(voices, voice_names):
    for voice in voices.voices:
        if voice.name in voice_names:
            return voice.voice_id
    return None

# Ustaw klucz API
set_api_key("77c2a2122bbfeb8e570f91ef3cb27982") # Brian - Slow speaking and deep
# set_api_key("33c6333781381542275a0fef3c844028") # Brian
# Pobierz listę głosów
voices = voices()

# Wyszukaj identyfikator głosu
voice_id = find_voice_id(voices, ['Brian', 'Brian - Slow speaking and deep'])

# Jeśli identyfikator głosu został znaleziony, użyj go do generowania dźwięku
if voice_id:
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    # Tutaj możesz użyć url do generowania dźwięku
    print(url)
else:
    print("Nie znaleziono głosu.")

In [ ]:
import asyncio
import aiohttp
from elevenlabs import voices as voices_func, set_api_key

async def find_voice_id(session, api_key, voice_names):
    # Ustaw klucz API
    set_api_key(api_key)

    # Pobierz listę głosów
    voices = voices_func()

    for voice in voices.voices:
        if voice.name in voice_names:
            return voice.voice_id

    return None

async def main():
    api_keys = ["77c2a2122bbfeb8e570f91ef3cb27982",
                "33c6333781381542275a0fef3c844028",
                "bf4e905fec0e3c7d7aa8710fa4493148",
                "d6ccf77426a0adf7b29612366f9cbd18",
                "59f3ff34e070b562f8ac4f5847bf5f77",
                "4d88c7168ddf8a163ffbf176276e72c5",
                "96028d102d0734fd815519ead6c6a625",
                "69629e2241ca3f96a3e97587ad5072e5","77c2a2122bbfeb8e570f91ef3cb27982",
                "33c6333781381542275a0fef3c844028",
                "bf4e905fec0e3c7d7aa8710fa4493148",
                "d6ccf77426a0adf7b29612366f9cbd18",
                "59f3ff34e070b562f8ac4f5847bf5f77",
                "4d88c7168ddf8a163ffbf176276e72c5",
                "96028d102d0734fd815519ead6c6a625",
                "69629e2241ca3f96a3e97587ad5072e5"
                ]
    voice_names = ['Brian', 'Brian - Slow speaking and deep']

    async with aiohttp.ClientSession() as session:
        tasks = [find_voice_id(session, api_key, voice_names) for api_key in api_keys]
        voice_ids = await asyncio.gather(*tasks)

        for api_key, voice_id in zip(api_keys, voice_ids):
            if voice_id:
                url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
                print(f"API Key: {api_key}, URL: {url}")
            else:
                print(f"API Key: {api_key}, Nie znaleziono głosu.")

# Uruchom główną funkcję asynchroniczną
await main()

## Przykładowe requests z API

In [ ]:
#@title Przykładowe requests z API

import requests
import rich

headers = {
    "Accept": "application/json",
    "xi-api-key": "69629e2241ca3f96a3e97587ad5072e5"
}

# Informacjie o wszystkich głosach
# Voices(
#     voices=[
#         Voice(
#             voice_id='21m00Tcm4TlvDq8ikWAM',
#             name='Rachel',
#             category='premade',
#             description=None,
#             labels={
#                 'accent': 'american',
#                 'description': 'calm',
#                 'age': 'young',
#                 'gender': 'female',
#                 'use case': 'narration'
#             },
#             samples=None,
#             settings=VoiceSettings(stability=0.5, similarity_boost=0.75),
#             design=None,
#             preview_url='https://storage.googleapis.com/eleven-public-prod/premade/voices/21m00Tcm4TlvDq8ikWAM/df67
# 88f9-5c96-470d-8312-aab3b3d8f50a.mp3'
#         ),

url = "https://api.elevenlabs.io/v1/voices/"
response = requests.get(url, headers=headers)

rich.print(response.text)




# Ustawienia domyśle głosu

# {"stability":0.5,"similarity_boost":0.5,"style":0.0,"use_speaker_boost":true}

url = "https://api.elevenlabs.io/v1/voices/{voice_id}/settings"
response = requests.get(url, headers=headers)

print(response.text)


# Ustawienia konta

# {"stability":0.5,"similarity_boost":0.5,"style":0.0,"use_speaker_boost":true}
# {"subscription":{"tier":"free","character_count":10000,"character_limit":10000,"can_extend_character_limit":false,"
# allowed_to_extend_character_limit":false,"next_character_count_reset_unix":1693860788,"voice_limit":3,"max_voice_ad
# d_edits":54,"voice_add_edit_counter":1,"professional_voice_limit":0,"can_extend_voice_limit":false,"can_use_instant
# _voice_cloning":false,"can_use_professional_voice_cloning":false,"currency":null,"status":"free"},"is_new_user":tru
# e,"xi_api_key":"69629e2241ca3f96a3e97587ad5072e5","can_use_delayed_payment_methods":false}


url = "https://api.elevenlabs.io/v1/user"
response = requests.get(url, headers=headers)

rich.print(response.text)


# Lista modeli i informacjie o nich

# [{"model_id":"eleven_multilingual_v2","name":"Eleven Multilingual
# v2","can_be_finetuned":true,"can_do_text_to_speech":true,"can_do_voice_conversion":true,"can_use_style":true,"can_u
# se_speaker_boost":true,"serves_pro_voices":false,"token_cost_factor":1.0,"description":"Our state of the art
# multilingual speech synthesis model, able to generate life-like speech in 28
# languages.","requires_alpha_access":false,"max_characters_request_free_user":2500,"max_characters_request_subscribe
# d_user":5000,"languages":[{"language_id":"en","name":"English"},{"language_id":"ja","name":"Japanese"},{"language_i
# d":"zh","name":"Chinese"},{"language_id":"de","name":"German"},{"language_id":"hi","name":"Hindi"},{"language_id":"
# fr","name":"French"},{"language_id":"ko","name":"Korean"},{"language_id":"pt","name":"Portuguese"},{"language_id":"
# it","name":"Italian"},{"language_id":"es","name":"Spanish"},{"language_id":"id","name":"Indonesian"},{"language_id"
# :"nl","name":"Dutch"},{"language_id":"tr","name":"Turkish"},{"language_id":"fil","name":"Filipino"},{"language_id":
# "pl","name":"Polish"},{"language_id":"sv","name":"Swedish"},{"language_id":"bg","name":"Bulgarian"},{"language_id":
# "ro","name":"Romanian"},{"language_id":"ar","name":"Arabic"},{"language_id":"cs","name":"Czech"},{"language_id":"el
# ","name":"Greek"},{"language_id":"fi","name":"Finnish"},{"language_id":"hr","name":"Croatian"},{"language_id":"ms",
# "name":"Malay"},{"language_id":"sk","name":"Slovak"},{"language_id":"da","name":"Danish"},{"language_id":"ta","name
# ":"Tamil"},{"language_id":"uk","name":"Ukrainian"}]},

url = "https://api.elevenlabs.io/v1/models"
response = requests.get(url, headers=headers)
rich.print(response.text)





## Text to Speech

In [ ]:
import ipywidgets as widgets
from google.colab import files
import IPython.display as ipd
import requests
import os
from IPython.display import display, HTML

# Ustawienia generowania dźwięku
CHUNK_SIZE = 2048
url = "https://api.elevenlabs.io/v1/text-to-speech/ZFeKTJIJXSBdBdyngreH"

headers = {
    "Accept": "audio/mpeg",
    "Content-Type": "application/json",
    "xi-api-key": "69629e2241ca3f96a3e97587ad5072e5"
}

# Interaktywne widgety do konfiguracji generowania dźwięku
text_widget = widgets.Textarea(
    value="OK",  # Domyślny tekst
    description="Tekst:",
    rows=5,
    layout=widgets.Layout(width="90%", height="150px"),
    max_chars=5000
)

model_id_widget = widgets.Dropdown(
    options=["eleven_monolingual_v1", "eleven_multilingual_v1", "eleven_multilingual_v2"],
    value="eleven_multilingual_v2",
    description="ID modelu:"
)

stability_widget = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=1.0,
    step=0.1,
    description="Stabilność:"
)

similarity_boost_widget = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=1.0,
    step=0.1,
    description="Podobność:"
)

# Funkcja do pobierania pliku audio
def download_audio(b):
    data = {
        "text": text_widget.value,
        "model_id": model_id_widget.value,
        "voice_settings": {
            "stability": stability_widget.value,
            "similarity_boost": similarity_boost_widget.value
        }
    }

    response = requests.post(url, json=data, headers=headers)

    counter = 1
    output_dir = 'output_audio'
    os.makedirs(output_dir, exist_ok=True)
    output_filename = create_unique_filename(output_dir, counter)

    with open(output_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
            if chunk:
                f.write(chunk)

    with output_audio_widget:
        display(HTML(f'<h3>Plik: {os.path.basename(output_filename)}</h3>'))  # Formatowanie jako nagłówek H3
        audio_display = ipd.Audio(output_filename)
        display(audio_display)

# Przycisk do pobierania pliku
button = widgets.Button(description="Wygeneruj Audio",
                        layout=widgets.Layout(width="300px", height="30px"))

output_audio_widget = widgets.Output()
output = widgets.Output()
button.on_click(download_audio)

# Wyświetlanie opisu, listy audio, podglądu i przycisków pobierania
display(
    HTML('<h1>ElevenLabs API Google Colab</h1>'),
    text_widget,
    model_id_widget,
    stability_widget,
    similarity_boost_widget,
    button,
    output_audio_widget,
    output
)


## Wyświetl Ustawienia Domyślne Danego Głosu

In [ ]:
import requests
import rich
url = "https://api.elevenlabs.io/v1/voices/ZFeKTJIJXSBdBdyngreH/settings"

headers = {
    "Accept": "application/json",
    "xi-api-key": "69629e2241ca3f96a3e97587ad5072e5"
}

response = requests.get(url, headers=headers)

print(response.text)


import requests

url = "https://api.elevenlabs.io/v1/user"

headers = {
    "Accept": "application/json",
    "xi-api-key": "69629e2241ca3f96a3e97587ad5072e5"
}

response = requests.get(url, headers=headers)

rich.print(response.text)


## Pobieranie Audio

In [ ]:
import ipywidgets as widgets
from google.colab import files
import IPython.display as ipd
import requests
import os
import zipfile
from IPython.display import display, HTML

# Filtruj i sortuj pliki audio alfabetycznie
audio_files = sorted([filename for filename in os.listdir(output_dir) if filename.endswith('.mp3')])

# Wybór wielu plików audio
selected_audio = widgets.SelectMultiple(
    options=audio_files,
    description='',
    disabled=False
)

# Opis wyboru audio
audio_selection_description = widgets.HTML('<h3>Zaznacz Audio Do Pobrania - (Ctrl / Shift + Click)</h3>')

# Widget do wyświetlania nazwy pliku i podglądu wybranych plików audio
output_audio_widget = widgets.Output()

# Funkcja do wyświetlania nazwy pliku i podglądu audio
def display_selected_audio(change):
    with output_audio_widget:
        output_audio_widget.clear_output()
        selected_files = selected_audio.value
        for filename in selected_files:
            audio_path = os.path.join(output_dir, filename)
            display(HTML(f'<h3>File: {filename}</h3>'))  # Formatowanie jako nagłówek H3
            audio_display = ipd.Audio(audio_path)
            display(audio_display)

selected_audio.observe(display_selected_audio, names='value')

download_selected_button = widgets.Button(description='Pobierz Zaznaczone',
                                          layout=widgets.Layout(width='300px', height='30px'))

download_all_button = widgets.Button(description='Pobierz Wszystko',
                                     layout=widgets.Layout(width='300px', height='30px'))

# Funkcja do pobierania zaznaczonych plików audio
def download_selected_audio(b):
    selected_files = selected_audio.value
    zip_filename = os.path.join(output_dir, 'selected_audio.zip')
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for filename in selected_files:
            audio_path = os.path.join(output_dir, filename)
            zipf.write(audio_path, os.path.basename(audio_path))
    files.download(zip_filename)

# Funkcja do pobierania wszystkich plików audio
def download_all_audio(b):
    zip_filename = os.path.join(output_dir, 'all_audio.zip')
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for filename in audio_files:
            audio_path = os.path.join(output_dir, filename)
            zipf.write(audio_path, os.path.basename(audio_path))
    files.download(zip_filename)

download_selected_button.on_click(download_selected_audio)
download_all_button.on_click(download_all_audio)

# Wyświetlanie opisu, listy audio, podglądu i przycisków pobierania
display(audio_selection_description, selected_audio, output_audio_widget, download_selected_button, download_all_button)
print()


# TEST ❌

In [ ]:
# @title ElevenLabs Infinite TTS API Generator v1 { run: "auto" }

import ipywidgets as widgets
import IPython.display as ipd
import requests
import os
import zipfile
# from elevenlabs import voices, set_api_key
from IPython.display import clear_output, display, HTML

# Domyślne ścieżki
default_path = "/content/"

# Sprawdź i przypisz None, jeśli zmienne nie istnieją
if 'google_drive_path' not in locals():
    google_drive_path = None
if 'github_repo_path' not in locals():
    github_repo_path = None

# Funkcja: Sprawdź dostępne ścieżki i zwróć listę
def get_available_paths():
    available_paths = []

    if os.path.exists(default_path):
        available_paths.append('Google Colab')
    if google_drive_path is not None:
        available_paths.append('Google Drive')
    if github_repo_path is not None:
        available_paths.append('GitHub')

    return available_paths

# Funkcja: Aktualizuj ścieżkę na podstawie wyboru
def update_folder(change):
    new_path = default_path

    if change.new == 'Google Drive' and google_drive_path is not None:
        new_path = google_drive_path
    if change.new == 'GitHub' and github_repo_path is not None:
        new_path = github_repo_path

    %cd $new_path

# Pobierz dostępne ścieżki
available_paths = get_available_paths()

# Dodaj opcję "Google Colab" do listy dostępnych ścieżek, jeśli nie istnieje
if 'Google Colab' not in available_paths:
    available_paths.append('Google Colab')

# Stwórz przycisk typu radio do wyboru folderu z dostępnymi ścieżkami
folder_radio = widgets.RadioButtons(
    options=available_paths,
    description='Lokalizacja:',
    disabled=False
)

# Uaktualnij ścieżkę na podstawie wyboru
folder_radio.observe(update_folder, 'value')

audio_files = []
CHUNK_SIZE = 2048

# Stwórz folder wyjściowy
output_dir = 'output_audio'
os.makedirs(output_dir, exist_ok=True)

api_key_widget = widgets.Dropdown(
    options=[(key['description'], key['key']) for key in api_keys],
    description="Klucz API:"
)

def set_api_and_refresh(change):
    try:
        selected_key = next(key for key in api_keys if key['key'] == api_key_widget.value)
        set_api_key(selected_key['key'])
        refresh_voices_and_clear_ui()
    except Exception as e:
        clear_output()
        display(HTML('<h1>ElevenLabs TTS API Google Colab</h1>'))
        display(api_key_widget)
        display(HTML(f'<p style="color: red;">Błąd: Nieprawidłowy klucz API</p>'))

api_key_widget.observe(set_api_and_refresh, names='value')

def refresh_voices_and_clear_ui():
    clear_output()
    display(HTML('<h1>ElevenLabs Infinite TTS API Generator</h1>'))
    set_api_key(api_key_widget.value)

    # Pobranie dostępnych głosów
    available_voices = voices()
    voice_options = sorted([
        (
            f"{voice.name} ({voice.category}) - {voice.labels['accent']}" + (f", {voice.labels['description']}" if 'description' in voice.labels else "") + f", {voice.labels['age']}, {voice.labels['gender']}" + (f", {voice.labels['use case']}" if 'use case' in voice.labels else ""),
            voice.voice_id
        )
        for voice in available_voices.voices
    ])

    selected_voice = widgets.Dropdown(
        options=voice_options,
        description="Głos:"
    )

    def update_url_and_headers(change):
        global url, headers
        url = f"https://api.elevenlabs.io/v1/text-to-speech/{selected_voice.value}"
        headers = {
            "Accept": "audio/mpeg",
            "Content-Type": "application/json",
            "xi-api-key": api_key_widget.value
        }

    selected_voice.observe(update_url_and_headers, names='value')
    update_url_and_headers(None)  # Inicjalizacja url i headers

    # Interaktywne widgety do konfiguracji generowania dźwięku
    text_widget = widgets.Textarea(
        value="OK",  # Domyślny tekst
        description="Tekst:",
        rows=5,
        layout=widgets.Layout(width="90%", height="150px"),
        max_chars=5000
    )

    model_id_widget = widgets.Dropdown(
        options=["eleven_monolingual_v1", "eleven_multilingual_v1", "eleven_multilingual_v2"],
        value="eleven_multilingual_v2",
        description="ID modelu:"
    )

    stability_widget = widgets.FloatSlider(
        value=0.5,
        min=0.0,
        max=1.0,
        step=0.01,  # Zmieniony krok suwaka na 0.01
        description="Stabilność:"
    )

    similarity_boost_widget = widgets.FloatSlider(
        value=0.75,
        min=0.0,
        max=1.0,
        step=0.01,  # Zmieniony krok suwaka na 0.01
        description="Podobność:"
    )

    style_widget = widgets.FloatSlider(
        value=0.0,
        min=0.0,
        max=1.0,
        step=0.01,  # Zmieniony krok suwaka na 0.01
        description="Styl:"
    )

    use_speaker_boost_widget = widgets.RadioButtons(
        options=[("Tak", True), ("Nie", False)],
        value=True,
        description="Przesilenie:"
    )

    # Funkcja do pobierania pliku audio
    def create_unique_filename(output_dir, counter):
        while True:
            filename = os.path.join(output_dir, f"{counter:05d}.mp3")
            if not os.path.exists(filename):
                return filename
            counter += 1

    audio_files = sorted([filename for filename in os.listdir('output_audio') if filename.endswith('.mp3')])

    def download_audio(b):
        global audio_files
        data = {
            "text": text_widget.value,
            "model_id": model_id_widget.value,
            "voice_settings": {
                "stability": stability_widget.value,
                "similarity_boost": similarity_boost_widget.value,
                "style": style_widget.value,
                "use_speaker_boost": use_speaker_boost_widget.value

            }
        }

        response = requests.post(url, json=data, headers=headers)

        output_filename = create_unique_filename(output_dir, len(audio_files) + 1)

        with open(output_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
                if chunk:
                    f.write(chunk)

        audio_files.append(os.path.basename(output_filename))
        selected_audio.options = audio_files
        selected_audio.value = [os.path.basename(output_filename)]

    button = widgets.Button(description="Wygeneruj Audio", layout=widgets.Layout(width="300px", height="30px"))
    output_audio_widget = widgets.Output()
    button.on_click(download_audio)

    # Wyświetlanie opisu, listy audio, podglądu i przycisków pobierania
    selected_audio = widgets.SelectMultiple(
        options=audio_files,
        description='',
        disabled=False
    )

    audio_selection_description = widgets.HTML('<h3>Zaznacz Audio Do Pobrania - (Ctrl / Shift + Click)</h3>')

    output_audio_widget = widgets.Output()

    def display_selected_audio(change):
        with output_audio_widget:
            output_audio_widget.clear_output()
            selected_files = selected_audio.value
            if len(selected_files) == 1:
                filename = selected_files[0]
                audio_path = os.path.join(output_dir, filename)
                display(HTML(f'<h3>Plik: {filename}</h3>'))
                audio_display = ipd.Audio(audio_path, autoplay=True)  # Automatyczne odtwarzanie
                display(audio_display)
            else:
                for filename in selected_files:
                    audio_path = os.path.join(output_dir, filename)
                    display(HTML(f'<h3>Plik: {filename}</h3>'))
                    audio_display = ipd.Audio(audio_path)
                    display(audio_display)

    selected_audio.observe(display_selected_audio, names='value')

    download_selected_button = widgets.Button(description='Pobierz Zaznaczone',
                                              layout=widgets.Layout(width='300px', height='30px'))

    download_all_button = widgets.Button(description='Pobierz Wszystko',
                                        layout=widgets.Layout(width='300px', height='30px'))

    def download_selected_audio(b):
        selected_files = selected_audio.value
        zip_filename = os.path.join(output_dir, 'selected_audio.zip')
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for filename in selected_files:
                audio_path = os.path.join(output_dir, filename)
                zipf.write(audio_path, os.path.basename(audio_path))
        files.download(zip_filename)

    def download_all_audio(b):
        zip_filename = os.path.join(output_dir, 'all_audio.zip')
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for filename in audio_files:
                audio_path = os.path.join(output_dir, filename)
                zipf.write(audio_path, os.path.basename(audio_path))
        files.download(zip_filename)

    download_selected_button.on_click(download_selected_audio)
    download_all_button.on_click(download_all_audio)

    # Wyświetlanie opisu, listy audio, podglądu i przycisków pobierania
    display(folder_radio,
            api_key_widget,
            selected_voice,
            text_widget,
            model_id_widget,
            stability_widget,
            similarity_boost_widget,
            style_widget,
            use_speaker_boost_widget,
            button,
            audio_selection_description,
            selected_audio,
            output_audio_widget,
            download_selected_button,
            download_all_button)

display(api_key_widget)
refresh_voices_and_clear_ui()

In [ ]:
# @title ElevenLabs Infinite TTS API Generator PODSTAWA { run: "auto" }

import ipywidgets as widgets
import IPython.display as ipd
import requests
import os
import zipfile
from elevenlabs import voices, set_api_key, Models
from IPython.display import clear_output, display, HTML

# Domyślne ścieżki
default_path = "/content/"

# Sprawdź i przypisz None, jeśli zmienne nie istnieją
if 'google_drive_path' not in locals():
    google_drive_path = None
if 'github_repo_path' not in locals():
    github_repo_path = None

# Funkcja: Sprawdź dostępne ścieżki i zwróć listę
def get_available_paths():
    available_paths = []

    if os.path.exists(default_path):
        available_paths.append('Google Colab')
    if google_drive_path is not None:
        available_paths.append('Google Drive')
    if github_repo_path is not None:
        available_paths.append('GitHub')

    return available_paths

# Pobierz dostępne ścieżki
available_paths = get_available_paths()

# Dodaj opcję "Google Colab" do listy dostępnych ścieżek, jeśli nie istnieje
if 'Google Colab' not in available_paths:
    available_paths.append('Google Colab')

# Stwórz przycisk typu radio do wyboru folderu z dostępnymi ścieżkami
folder_radio = widgets.RadioButtons(
    options=available_paths,
    description='Lokalizacja:',
    disabled=False
)

# Funkcja: Aktualizuj ścieżkę na podstawie wyboru
def update_folder(change):
    new_path = default_path

    if change.new == 'Google Drive' and google_drive_path is not None:
        new_path = google_drive_path
    if change.new == 'GitHub' and github_repo_path is not None:
        new_path = github_repo_path

    %cd $new_path
    refresh_output_audio_list()

# Uaktualnij ścieżkę na podstawie wyboru
folder_radio.observe(update_folder, 'value')


# Stwórz folder wyjściowy
output_dir = 'output_audio'
os.makedirs(output_dir, exist_ok=True)

api_key_widget = widgets.Dropdown(
    options=[(key['description'], key['key']) for key in api_keys],
    description="Klucz API:"
)


def set_api_and_refresh(change):
    try:
        selected_key = next(
            key for key in api_keys if key['key'] == api_key_widget.value)
        set_api_key(selected_key['key'])
        refresh_voices_and_clear_ui()
    except Exception as e:
        clear_output()
        display(HTML('<h1>ElevenLabs TTS API Google Colab</h1>'))
        display(api_key_widget)
        display(HTML('<p style="color: red;">Błąd: Nieprawidłowy klucz API</p>'))


api_key_widget.observe(set_api_and_refresh, names='value')


audio_files = []
CHUNK_SIZE = 2048


def refresh_voices_and_clear_ui():
    clear_output()
    display(HTML('<h1>ElevenLabs Infinite TTS API Generator</h1>'))
    set_api_key(api_key_widget.value)

    # Pobranie dostępnych głosów
    available_voices = voices()
    voice_options = sorted([
        (
            f"{voice.name} ({voice.category}) - {voice.labels['accent']}" + (f", {voice.labels['description']}" if 'description' in voice.labels else "") +
            f", {voice.labels['age']}, {voice.labels['gender']}" +
            (f", {voice.labels['use case']}" if 'use case' in voice.labels else ""),
            voice.voice_id
        )
        for voice in available_voices.voices
    ])

    selected_voice = widgets.Dropdown(
        options=voice_options,
        description="Głos:"
    )

    def update_url_and_headers(change):
        global url, headers
        url = f"https://api.elevenlabs.io/v1/text-to-speech/{selected_voice.value}"
        headers = {
            "Accept": "audio/mpeg",
            "Content-Type": "application/json",
            "xi-api-key": api_key_widget.value
        }

    selected_voice.observe(update_url_and_headers, names='value')
    update_url_and_headers(None)  # Inicjalizacja url i headers

    # Interaktywne widgety do konfiguracji generowania dźwięku
    text_widget = widgets.Textarea(
        value="OK",  # Domyślny tekst
        description="Tekst:",
        rows=5,
        layout=widgets.Layout(width="90%", height="150px"),
        max_chars=5000
    )

    # Pobranie listy modeli
    models_list = Models.from_api()

    # Tworzenie mapowania nazw modeli na ich identyfikatory
    model_name_to_id = {
        model.name: model.model_id for model in models_list.models}

    # Tworzenie rozwijanej listy (dropdown) z nazwami modeli
    model_name_widget = widgets.Dropdown(
        # Przypisanie listy nazw jako opcje
        options=[model.name for model in models_list.models],
        value=models_list.models[0].name,  # Domyślnie wybrana nazwa modelu
        description="Model:"
    )

    stability_widget = widgets.FloatSlider(
        value=0.5,
        min=0.0,
        max=1.0,
        step=0.01,  # Zmieniony krok suwaka na 0.01
        description="Stabilność:"
    )

    similarity_boost_widget = widgets.FloatSlider(
        value=0.75,
        min=0.0,
        max=1.0,
        step=0.01,  # Zmieniony krok suwaka na 0.01
        description="Podobność:"
    )

    style_widget = widgets.FloatSlider(
        value=0.0,
        min=0.0,
        max=1.0,
        step=0.01,  # Zmieniony krok suwaka na 0.01
        description="Styl:"
    )

    use_speaker_boost_widget = widgets.RadioButtons(
        options=[("Tak", True), ("Nie", False)],
        value=True,
        description="Przesilenie:"
    )

    # Funkcja do pobierania pliku audio
    def create_unique_filename(output_dir, counter):
        while True:
            filename = os.path.join(output_dir, f"{counter:05d}.mp3")
            if not os.path.exists(filename):
                return filename
            counter += 1

    def refresh_output_audio_list():
        global audio_files
        audio_files = sorted([filename for filename in os.listdir(
            output_dir) if filename.endswith('.mp3')])

    def download_audio(b):
        refresh_output_audio_list()
        data = {
            "text": text_widget.value,
            "model_id": model_name_to_id[model_name_widget.value],
            "voice_settings": {
                "stability": stability_widget.value,
                "similarity_boost": similarity_boost_widget.value,
                "style": style_widget.value,
                "use_speaker_boost": use_speaker_boost_widget.value

            }
        }

        response = requests.post(url, json=data, headers=headers)

        output_filename = create_unique_filename(
            output_dir, len(audio_files) + 1)

        with open(output_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
                if chunk:
                    f.write(chunk)

        audio_files.append(os.path.basename(output_filename))
        selected_audio.options = audio_files
        selected_audio.value = [os.path.basename(output_filename)]

    button = widgets.Button(description="Wygeneruj Audio",
                            layout=widgets.Layout(width="300px", height="30px"))
    output_audio_widget = widgets.Output()
    button.on_click(download_audio)

    # Wyświetlanie opisu, listy audio, podglądu i przycisków pobierania
    selected_audio = widgets.SelectMultiple(
        options=audio_files,
        description='',
        disabled=False
    )

    audio_selection_description = widgets.HTML(
        '<h3>Zaznacz Audio Do Pobrania - (Ctrl / Shift + Click)</h3>')

    output_audio_widget = widgets.Output()

    def display_selected_audio(change):
        with output_audio_widget:
            output_audio_widget.clear_output()
            selected_files = selected_audio.value
            if len(selected_files) == 1:
                filename = selected_files[0]
                audio_path = os.path.join(output_dir, filename)
                display(HTML(f'<h3>Plik: {filename}</h3>'))
                # Automatyczne odtwarzanie
                audio_display = ipd.Audio(audio_path, autoplay=True)
                display(audio_display)
            else:
                for filename in selected_files:
                    audio_path = os.path.join(output_dir, filename)
                    display(HTML(f'<h3>Plik: {filename}</h3>'))
                    audio_display = ipd.Audio(audio_path)
                    display(audio_display)

    selected_audio.observe(display_selected_audio, names='value')

    download_selected_button = widgets.Button(description='Pobierz Zaznaczone',
                                              layout=widgets.Layout(width='300px', height='30px'))

    download_all_button = widgets.Button(description='Pobierz Wszystko',
                                         layout=widgets.Layout(width='300px', height='30px'))

    def download_selected_audio(b):
        selected_files = selected_audio.value
        zip_filename = os.path.join(output_dir, 'selected_audio.zip')
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for filename in selected_files:
                audio_path = os.path.join(output_dir, filename)
                zipf.write(audio_path, os.path.basename(audio_path))
        files.download(zip_filename)

    def download_all_audio(b):
        zip_filename = os.path.join(output_dir, 'all_audio.zip')
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for filename in audio_files:
                audio_path = os.path.join(output_dir, filename)
                zipf.write(audio_path, os.path.basename(audio_path))
        files.download(zip_filename)

    download_selected_button.on_click(download_selected_audio)
    download_all_button.on_click(download_all_audio)

    # Wyświetlanie opisu, listy audio, podglądu i przycisków pobierania
    display(folder_radio,
            api_key_widget,
            selected_voice,
            text_widget,
            model_name_widget,
            stability_widget,
            similarity_boost_widget,
            style_widget,
            use_speaker_boost_widget,
            button,
            audio_selection_description,
            selected_audio,
            output_audio_widget,
            download_selected_button,
            download_all_button)


display(api_key_widget)
refresh_voices_and_clear_ui()


In [ ]:
# https://ipywidgets.readthedocs.io/en/stable/

# Ustawienie klucza API
#@title Wpisz swój klucz API { run: "auto" }
API = "96028d102d0734fd815519ead6c6a625" #@param {type:"string"}
DISCRIPTION = "mateuszmroz001@gmail.com" #@param {type:"string"}

api_keys = [
    {"key": "96028d102d0734fd815519ead6c6a625", "description": "mateuszmroz001@gmail.com"},
    {"key": "69629e2241ca3f96a3e97587ad5072e5", "description": "eqdlaec669@fugmi.com"},
    {"key": "klucz_api_3", "description": "Opis klucza API 3"},
    # Dodaj więcej kluczy API w podobny sposób
]





In [ ]:
# @title Przykład ze strony { run: "auto" }

# import requests

# # streaming chunk size
# CHUNK_SIZE = 1024

# XI_API_KEY = "<xi-api-key>"
# VOICE_SAMPLE_PATH1 = "<path-to-file>"
# VOICE_SAMPLE_PATH2 = "<path-to-file>"
# OUTPUT_PATH = "<path-to-file>"

# add_voice_url = "https://api.elevenlabs.io/v1/voices/add"

# headers = {
#   "Accept": "application/json",
#   "xi-api-key": XI_API_KEY
# }

# data = {
#     'name': 'Voice name',
#     'labels': '{"accent": "American", "gender": "Female"}',
#     'description': 'An old American male voice with a slight hoarseness in his throat. Perfect for news.'
# }

# files = [
#     ('files', ('sample1.mp3', open(VOICE_SAMPLE_PATH1, 'rb'), 'audio/mpeg')),
#     ('files', ('sample2.mp3', open(VOICE_SAMPLE_PATH2, 'rb'), 'audio/mpeg'))
# ]

# response = requests.post(add_voice_url, headers=headers, data=data, files=files)
# voice_id = response.json()["voice_id"]

# # get default voice settings
# response = requests.get(
#     "https://api.elevenlabs.io/v1/voices/settings/default",
#     headers={ "Accept": "application/json" }
# ).json()
# stability, similarity_boost = response["stability"], response["similarity_boost"]

# tts_url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}/stream"

# headers["Content-Type"] = "application/json"

# data = {
#   "text": "Some very long text to be read by the voice",
#   "model_id": "eleven_monolingual_v1",
#   "voice_settings": {
#     "stability": stability,
#     "similarity_boost": similarity_boost
#   }
# }

# response = requests.post(tts_url, json=data, headers=headers, stream=True)

# with open(OUTPUT_PATH, 'wb') as f:
#     for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
#         if chunk:
#             f.write(chunk)

# # Retrieve history. It should contain generated sample.
# history_url = "https://api.elevenlabs.io/v1/history"

# headers = {
#   "Accept": "application/json",
#   "xi-api-key": XI_API_KEY
# }

# response = requests.get(history_url, headers=headers)

# print(response.text)





# from elevenlabs import clone, generate, play, set_api_key
# from elevenlabs.api import History

# set_api_key("<YOUR_API_KEY>")

# voice = clone(
#     name="Voice Name",
#     description="An old American male voice with a slight hoarseness in his throat. Perfect for news.",
#     files=["./sample1.mp3", "./sample2.mp3"],
# )

# audio = generate(text="Some very long text to be read by the voice", voice=voice)

# play(audio)

# history = History.from_api()
# print(history)




# # zamiana tekstu na mowe
# import requests

# CHUNK_SIZE = 1024
# url = "https://api.elevenlabs.io/v1/text-to-speech/<voice-id>"

# headers = {
#   "Accept": "audio/mpeg",
#   "Content-Type": "application/json",
#   "xi-api-key": "<xi-api-key>"
# }

# data = {
#   "text": "Hi! My name is Bella, nice to meet you!",
#   "model_id": "eleven_monolingual_v1",
#   "voice_settings": {
#     "stability": 0.5,
#     "similarity_boost": 0.5
#   }
# }

# response = requests.post(url, json=data, headers=headers)
# with open('output.mp3', 'wb') as f:
#     for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
#         if chunk:
#             f.write(chunk)



# strumieniuj tekst na mowe

import requests

CHUNK_SIZE = 1024
url = "https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL/stream"

headers = {
  "Accept": "audio/mpeg",
  "Content-Type": "application/json",
  "xi-api-key": "4d88c7168ddf8a163ffbf176276e72c5"
}

data = {
  "text": "Hi! My name is Bella, nice to meet you!",
  "model_id": "eleven_monolingual_v1",
  "voice_settings": {
    "stability": 0.5,
    "similarity_boost": 0.5
  }
}

response = requests.post(url, json=data, headers=headers, stream=True)

with open('output.mp3', 'wb') as f:
    for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)





In [ ]:
# @title BUTTON { run: "auto" }

execute_button = widgets.Button(
    description='Wykonaj',
    style=dict(font_weight='bold'),
    layout=widgets.Layout(width='300px', height='30px')
)


In [ ]:
#@title Komentarze w GC

import ipywidgets as widgets

# @title: Tytuł Notatnika
# Ustawia tytuł notatnika.
# @title Przykładowy Tytuł

# @markdown: Tekst w formie Markdown
# Pozwala na wstawienie tekstu w formie Markdown do notatnika.
# Przykład:
# @markdown ## Instrukcje

# Ten notatnik zawiera przykładowy tekst w formie Markdown.

# @param: Interaktywny parametr
# Tworzy interaktywny parametr, który użytkownik może dostosować.
# Przykład:
@param {type:"slider", min:0, max:100, step:1, default:50} wartość_slidera
wartość_slidera = 26 # @param {type:"slider", min:0, max:100, step:1, default:50}




## DZIAŁA

In [ ]:
# @title 2.1 Dodaj / Usuń Klucz API w Google Drive { run: "auto" }

# Importowanie niezbędnych modułów
import os
import json
import ipywidgets as widgets
from IPython.display import display
import requests

# Ścieżka do katalogu i pliku JSON przechowującego klucze API
directory_path = '/content/drive/MyDrive/11Labs_TTS_Colab/'
file_name = 'elevenlabs_api_keys.json'
file_path = os.path.join(directory_path, file_name)

# Funkcja do wczytywania kluczy API z pliku
def load_api_keys():
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return []

# Funkcja do zapisu kluczy API do pliku
def save_api_keys(keys):
    with open(file_path, 'w') as file:
        # Dodanie argumentu 'indent' do formatowania
        json.dump(keys, file, indent=4)

# Funkcja do wywołania API i obliczenia pozostałych znaków
def call_api_and_calculate_remaining_characters(api_key):
    try:
        url = "https://api.elevenlabs.io/v1/user"
        headers = {
            "Accept": "application/json",
            "xi-api-key": api_key
        }
        response = requests.get(url, headers=headers)
        response_data = response.json()
        character_limit = response_data['subscription']['character_limit']
        character_count = response_data['subscription']['character_count']
        return character_limit - character_count
    except (requests.exceptions.RequestException, KeyError, json.JSONDecodeError) as e:
        print("Wystąpił błąd podczas wywoływania API:", e)
        return None

# Obszar tekstowy do wyświetlania informacji o wybranym kluczu
key_info_output = widgets.Output()

# Wczytanie istniejących kluczy API
api_keys = load_api_keys()

# Funkcja do wyświetlania informacji o wybranym kluczu w obszarze tekstowym
def show_key_info(selected_key):
    key_info_output.clear_output()
    with key_info_output:
        if selected_key is not None:  # Dodano warunek sprawdzający, czy selected_key nie jest None
            print(f"ID: {selected_key['id']}")
            print(f"Key: {selected_key['key']}")
            print(f"Description: {selected_key['description']}")
            if 'remaining_characters' in selected_key:
                print(
                    f"Remaining Characters: {selected_key['remaining_characters']}")

# Funkcja aktualizująca i zapisująca pozostałe znaki do pliku JSON po wywołaniu API
def update_and_save_remaining_characters(api_key):
    if api_key is None:
        return
    try:
        remaining_characters = call_api_and_calculate_remaining_characters(
            api_key)
        if remaining_characters is not None:
            for key in api_keys:
                if key['key'] == api_key:
                    key['remaining_characters'] = remaining_characters
                    break
            save_api_keys(api_keys)
            # Aktualizacja informacji o wybranym kluczu
            show_key_info(key_info_dropdown.value)
    except Exception as e:
        print("Wystąpił błąd podczas aktualizowania pozostałych znaków:", e)


# Funkcja do generowania nowego ID
def generate_new_id(api_keys):
    return max(key['id'] for key in api_keys) + 1 if api_keys else 1


# Tworzenie interaktywnego przycisku wyboru akcji: Dodaj klucz API / Usuń klucz API
action_radio = widgets.RadioButtons(
    options=['Dodaj klucz API', 'Usuń klucz API'],
    description='Akcja:'
)

# Tworzenie rozwijanej listy opcji dla istniejących kluczy API
key_info_dropdown = widgets.Dropdown(
    options=[(f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key)
             for key in api_keys],
    description='Wybierz:',
    value=api_keys[0] if api_keys else None
)

# Obszar tekstowy do wyświetlania informacji o wybranym kluczu
key_info_output = widgets.Output()

# Iteracja przez istniejące klucze API i aktualizacja pozostałych znaków
for key in api_keys:
    update_and_save_remaining_characters(key['key'])

# Funkcja wywoływana po zmianie wyboru w rozwijanej liście
def key_info_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] in api_keys:
            selected_key = change['new']
            show_key_info(selected_key)
            if selected_key is not None:
                # Aktualizacja pozostałych znaków po zmianie klucza
                update_and_save_remaining_characters(selected_key['key'])

                # Aktualizacja listy rozwijanej po wyborze klucza
                key_info_dropdown.options = [
                    (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in api_keys]
                key_info_dropdown.value = selected_key  # Ustawienie wybranego klucza

key_info_dropdown.observe(key_info_dropdown_change)

# Przycisk do wykonania wybranej akcji
execute_button = widgets.Button(
    description='Wykonaj',
    style=dict(font_weight='bold'),
    layout=widgets.Layout(width='300px', height='30px')
)

# Funkcja wywoływana po kliknięciu przycisku "Wykonaj"
def execute_action(button):
    selected_action = action_radio.value
    if selected_action == 'Dodaj klucz API':
        new_key = new_key_text.value
        new_description = new_description_text.value
        if new_key and new_description:
            try:
                # Sprawdzenie, czy nowy klucz API lub opis nie są już w użyciu
                if any(key['key'] == new_key or key['description'] == new_description for key in api_keys):
                    print(
                        "Klucz API lub opis się powtarzają. Proszę wybrać unikalne wartości.")
                else:
                    new_id = generate_new_id(api_keys)
                    api_keys.append(
                        {"id": new_id, "key": new_key, "description": new_description})
                    save_api_keys(api_keys)
                    print("Nowy klucz API został dodany.")
                    key_info_dropdown.options = [
                        (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in api_keys]

                    # Znalezienie i zaznaczenie ostatnio dodanego klucza w opcjach rozwijanej listy
                    for option in key_info_dropdown.options:
                        if option[1]['id'] == new_id:
                            key_info_dropdown.value = option[1]
                            break

                    # Aktualizacja pozostałych znaków po dodaniu nowego klucza
                    update_and_save_remaining_characters(new_key)

                    # Odświeżanie ilości znaków dla wybranego klucza
                    selected_key = key_info_dropdown.value
                    if selected_key is not None:
                        update_and_save_remaining_characters(
                            selected_key['key'])
            except Exception as e:
                print("Wystąpił błąd podczas dodawania klucza API:", e)
    elif selected_action == 'Usuń klucz API':
        if key_info_dropdown.value is not None:
            try:
                selected_key = key_info_dropdown.value
                key_id = selected_key['id']
                for key in api_keys:
                    if key['id'] == key_id:
                        api_keys.remove(key)
                        save_api_keys(api_keys)
                        print(f"Klucz API o ID '{key_id}' został usunięty.")

                        # Aktualizacja opcji rozwijanej listy po usunięciu klucza
                        key_info_dropdown.options = [
                            (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in api_keys]

                        # Ustawienie selected_key na None po usunięciu klucza
                        key_info_dropdown.value = None
                        key_info_output.clear_output()  # Wyczyść obszar informacji o kluczu

                        break
            except Exception as e:
                print("Wystąpił błąd podczas usuwania klucza API:", e)
        else:
            print("Wybierz klucz do usunięcia.")

# Przypisanie funkcji do odpowiednich akcji po kliknięciu przycisku
execute_button.on_click(execute_action)

# Wyświetlenie interaktywnych widgetów
display(action_radio)
new_key_text = widgets.Text(description='Klucz API:')
new_description_text = widgets.Text(description='Opis:')
display(new_key_text)
display(new_description_text)
display(key_info_dropdown)
display(key_info_output)
display(execute_button)

# [
#     {
#         "id": 1,
#         "key": "69629e2241ca3f96a3e97587ad5072e5",
#         "description": "eqdlaec669@fugmi.com",
#         "remaining_characters": 376
#     },
#     {
#         "id": 2,
#         "key": "96028d102d0734fd815519ead6c6a625",
#         "description": "mateuszmroz001@gmail.com",
#         "remaining_characters": 4859
#     },
#     {
#         "id": 3,
#         "key": "4d88c7168ddf8a163ffbf176276e72c5",
#         "description": "mateuszmrozkun@gmail.com",
#         "remaining_characters": 10000
#     },
#     {
#         "id": 4,
#         "key": "59f3ff34e070b562f8ac4f5847bf5f77",
#         "description": "tojestpoprostutest@gmail.com",
#         "remaining_characters": 10000
#     }
# ]


In [ ]:
# @title 2.2 Dodaj / Usuń / Zapisz Klucze API Oraz Pliki Audio Do Repozytorium GitHub { run: "auto" }
import os
import json
import ipywidgets as widgets
from IPython.display import display
import requests
import subprocess

# Twój Personal Access Token
token = token

# Adres URL repozytorium
repo_url = repo_url

# Ścieżka do folderu, w którym znajduje się sklonowane repozytorium
directory_path = github_repo_path

# Konstrukcja URL z tokenem autoryzacyjnym
auth_repo_url = auth_repo_url

# Plik z kluczami API
file_name = 'elevenlabs_api_keys.json'
file_path = os.path.join(directory_path, file_name)

# Funkcja do aktualizacji repozytorium na GitHubie
def update_github_repo(commit_msg):
    try:
        # Przejście do odpowiedniego folderu
        %cd {directory_path}
        # print(f"Przejście do: {directory_path}")

        # Ustawienie konfiguracji dla Git
        # global_email = "mateuszmroz@gmail.com"  # Zaktualizuj na swoją wartość
        # global_name = "MattyMroz"  # Zaktualizuj na swoją wartość
        # !git config --global user.email "{global_email}"
        # !git config --global user.name "{global_name}"
        # print("Konfiguracja użytkownika Git zakończona!")

        # Dodawanie, commitowanie i wysyłanie zmian
        print("Wysyłanie zmian do repozytorium:")
        !git add .
        !git commit -m "{commit_msg}"
        !git push origin main
        print("Repozytorium zostało zaktualizowane na GitHubie.")
    except Exception as e:
        print("Wystąpił błąd podczas aktualizowania repozytorium:", e)

# Funkcja do wczytywania kluczy API z pliku
def load_api_keys():
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return []

# Funkcja do zapisu kluczy API do pliku
def save_api_keys(keys):
    with open(file_path, 'w') as file:
        # Dodanie argumentu 'indent' do formatowania
        json.dump(keys, file, indent=4)

# Funkcja do wywołania API i obliczenia pozostałych znaków
def call_api_and_calculate_remaining_characters(api_key):
    try:
        url = "https://api.elevenlabs.io/v1/user"
        headers = {
            "Accept": "application/json",
            "xi-api-key": api_key
        }
        response = requests.get(url, headers=headers)
        response_data = response.json()
        character_limit = response_data['subscription']['character_limit']
        character_count = response_data['subscription']['character_count']
        return character_limit - character_count
    except (requests.exceptions.RequestException, KeyError, json.JSONDecodeError) as e:
        print("Wystąpił błąd podczas wywoływania API:", e)
        return None

# Wczytanie istniejących kluczy API
api_keys = load_api_keys()

# Funkcja do wyświetlania informacji o wybranym kluczu w obszarze tekstowym
def show_key_info(selected_key):
    key_info_output.clear_output()
    with key_info_output:
        if selected_key is not None:  # Dodano warunek sprawdzający, czy selected_key nie jest None
            print(f"ID: {selected_key['id']}")
            print(f"Key: {selected_key['key']}")
            print(f"Description: {selected_key['description']}")
            if 'remaining_characters' in selected_key:
                print(
                    f"Remaining Characters: {selected_key['remaining_characters']}")

# Funkcja aktualizująca i zapisująca pozostałe znaki do pliku JSON po wywołaniu API
def update_and_save_remaining_characters(api_key):
    if api_key is None:
        return
    try:
        remaining_characters = call_api_and_calculate_remaining_characters(
            api_key)
        if remaining_characters is not None:
            for key in api_keys:
                if key['key'] == api_key:
                    key['remaining_characters'] = remaining_characters
                    break
            save_api_keys(api_keys)
            # Aktualizacja informacji o wybranym kluczu
            show_key_info(key_info_dropdown.value)
    except Exception as e:
        print("Wystąpił błąd podczas aktualizowania pozostałych znaków:", e)

# Funkcja do generowania nowego ID
def generate_new_id(api_keys):
    return max(key['id'] for key in api_keys) + 1 if api_keys else 1

# Tworzenie interaktywnego przycisku wyboru akcji: Dodaj klucz API / Usuń klucz API
action_radio = widgets.RadioButtons(
    options=['Dodaj klucz API', 'Usuń klucz API'],
    description='Akcja:'
)

# Tworzenie rozwijanej listy opcji dla istniejących kluczy API
key_info_dropdown = widgets.Dropdown(
    options=[(f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key)
             for key in api_keys],
    description='Wybierz:',
    value=api_keys[0] if api_keys else None
)

# Obszar tekstowy do wyświetlania informacji o wybranym kluczu
key_info_output = widgets.Output()

# Iteracja przez istniejące klucze API i aktualizacja pozostałych znaków
for key in api_keys:
    update_and_save_remaining_characters(key['key'])

# Funkcja wywoływana po zmianie wyboru w rozwijanej liście
def key_info_dropdown_change(change):
    if change['new'] in api_keys:
        selected_key = change['new']
        show_key_info(selected_key)
        if selected_key is not None:
            # Aktualizacja pozostałych znaków po zmianie klucza
            update_and_save_remaining_characters(selected_key['key'])
            update_github_repo("Zaktualizowano dane klucza API")

            # Aktualizacja listy rozwijanej po wyborze klucza
            key_info_dropdown.options = [
                (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in api_keys]
            key_info_dropdown.value = selected_key  # Ustawienie wybranego klucza

key_info_dropdown.observe(key_info_dropdown_change)

# Przycisk do wykonania wybranej akcji
execute_button = widgets.Button(
    description='Wykonaj',
    style=dict(font_weight='bold'),
    layout=widgets.Layout(width='300px', height='30px')
)

# Funkcja wywoływana po kliknięciu przycisku "Wykonaj"
def execute_action(button):
    selected_action = action_radio.value
    if selected_action == 'Dodaj klucz API':
        new_key = new_key_text.value
        new_description = new_description_text.value
        if new_key and new_description:
            try:
                # Sprawdzenie, czy nowy klucz API lub opis nie są już w użyciu
                if any(key['key'] == new_key or key['description'] == new_description for key in api_keys):
                    print(
                        "Klucz API lub opis się powtarzają. Proszę wybrać unikalne wartości.")
                else:
                    new_id = generate_new_id(api_keys)
                    api_keys.append(
                        {"id": new_id, "key": new_key, "description": new_description})
                    save_api_keys(api_keys)
                    print("Nowy klucz API został dodany.")
                    key_info_dropdown.options = [
                        (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in api_keys]

                    # Znalezienie i zaznaczenie ostatnio dodanego klucza w opcjach rozwijanej listy
                    for option in key_info_dropdown.options:
                        if option[1]['id'] == new_id:
                            key_info_dropdown.value = option[1]
                            break

                    # Aktualizacja pozostałych znaków po dodaniu nowego klucza
                    update_and_save_remaining_characters(new_key)

                    # Odświeżanie ilości znaków dla wybranego klucza
                    selected_key = key_info_dropdown.value
                    if selected_key is not None:
                        update_and_save_remaining_characters(
                            selected_key['key'])
                update_github_repo("Dodano klucz API")
            except Exception as e:
                print("Wystąpił błąd podczas dodawania klucza API:", e)
    elif selected_action == 'Usuń klucz API':
        if key_info_dropdown.value is not None:
            try:
                selected_key = key_info_dropdown.value
                key_id = selected_key['id']
                for key in api_keys:
                    if key['id'] == key_id:
                        api_keys.remove(key)
                        save_api_keys(api_keys)
                        print(f"Klucz API o ID '{key_id}' został usunięty.")

                        # Aktualizacja opcji rozwijanej listy po usunięciu klucza
                        key_info_dropdown.options = [
                            (f"ID: {key['id']}, Desc: {key['description']}, RC: {key.get('remaining_characters', 'N/A')}", key) for key in api_keys]

                        # Ustawienie selected_key na None po usunięciu klucza
                        key_info_dropdown.value = None
                        key_info_output.clear_output()  # Wyczyść obszar informacji o kluczu

                        break
                update_github_repo("Usunięto klucz API")
            except Exception as e:
                print("Wystąpił błąd podczas usuwania klucza API:", e)
        else:
            print("Wybierz klucz do usunięcia.")

# Przypisanie funkcji do odpowiednich akcji po kliknięciu przycisku
execute_button.on_click(execute_action)


# Wyświetlenie interaktywnych widgetów
display(action_radio)
new_key_text = widgets.Text(description='Klucz API:')
new_description_text = widgets.Text(description='Opis:')
display(new_key_text)
display(new_description_text)
display(key_info_dropdown)
display(key_info_output)
display(execute_button)

# Aktualizacja repozytorium na GitHubie z odpowiednim komunikatem
update_github_repo("Automatyczna aktualizacja pozostałych znaków")

# [
#     {
#         "id": 1,
#         "key": "69629e2241ca3f96a3e97587ad5072e5",
#         "description": "eqdlaec669@fugmi.com",
#         "remaining_characters": 376
#     },
#     {
#         "id": 2,
#         "key": "96028d102d0734fd815519ead6c6a625",
#         "description": "mateuszmroz001@gmail.com",
#         "remaining_characters": 4859
#     },
#     {
#         "id": 3,
#         "key": "4d88c7168ddf8a163ffbf176276e72c5",
#         "description": "mateuszmrozkun@gmail.com",
#         "remaining_characters": 10000
#     },
#     {
#         "id": 4,
#         "key": "59f3ff34e070b562f8ac4f5847bf5f77",
#         "description": "tojestpoprostutest@gmail.com",
#         "remaining_characters": 10000
#     }
# ]


In [ ]:
# @title ElevenLabs Infinite TTS API - Anime Lektor PL #1 bez asynchroniczności wolniejsze { run: "auto" }
import pysrt
from pydub import AudioSegment
import os
import glob
from google.colab import files
import ipywidgets as widgets
import requests
from IPython.display import display, HTML
import shutil

#===============================================================================
# Button CSS
display(HTML('''
<style>
    .widget-button {
        font-weight: bold !important;
    }
    .widget-button.jupyter-widgets.widget-button.mod-active,
    .widget-button:active {
        background-color: #2d2e2e;
    }
    .widget-toggle-button {
        font-weight: bold !important;
    }
    .widget-toggle-button.jupyter-widgets.widget-toggle-button.mod-active,
    .widget-toggle-button:active {
        background-color: #2d2e2e;
    }
</style>
'''))
#===============================================================================

os.chdir('/content/')

class AudioGenerator:
    def __init__(self, input_folder='input', output_folder='audio_output'):
        self.api_keys = []
        self.api_key_statuses = {key: True for key in self.api_keys}
        self.audio_cache = {}
        self.use_cache = False  # Przełącznik cache'a

        self.input_folder = input_folder
        self.output_folder = output_folder

        os.makedirs(input_folder, exist_ok=True)
        os.makedirs(output_folder, exist_ok=True)

    def update_api_keys(self, api_keys):
        self.api_keys = [key['key'] for key in api_keys]
        self.api_key_statuses = {key['key']: True for key in api_keys}

    def generate_audio(self, text, index, api_key, temp_folder):
        CHUNK_SIZE = 1024
        url = "https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL" # Bella 0.5 0.75 | Stabilny Najlepszy
        # url = "https://api.elevenlabs.io/v1/text-to-speech/ExUL9WhYaSyp0aHiA3TU" # Scarlett - 💯% seductive 🔥 0.5 0.5 | Niestabilny
        # url = "https://api.elevenlabs.io/v1/text-to-speech/JoeAjOpsgvMdLp28p97A" # Brian - Slow speaking and deep 0.5 0.75 | Bardzo Stabilny
        # url = "https://api.elevenlabs.io/v1/text-to-speech/XgjAPAgn6XVf0WYFCReM" # Knightley - dapper and deep narrator 0.5 0.75 | Lekko Niestabilny
        # url = "https://api.elevenlabs.io/v1/text-to-speech/Vms109RQ0LiOrqV6Ktn9" # Joanne - pensive, introspective, soft and lovely  0.5 0.75 | Stabilny Monotonny
        # url = "https://api.elevenlabs.io/v1/text-to-speech/piTKgcLEGmPE4e6mEKli" # Nicole - szpt 0.5 0.75 | Lekko Niestabilny

        headers = {
            "Accept": "audio/mpeg",
            "Content-Type": "application/json",
            "xi-api-key": api_key
        }
        data = {
            "text": text,
            "model_id": "eleven_multilingual_v2",
            "voice_settings": {
                "stability": 0.5,
                "similarity_boost": 0.75
            }
        }

        audio_file_path = f'{temp_folder}/{index}.mp3'

        if self.use_cache and text in self.audio_cache:
            return self.audio_cache[text], True

        response = requests.post(url, json=data, headers=headers)
        if response.status_code == 200:
            with open(audio_file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
                    if chunk:
                        f.write(chunk)
            self.audio_cache[text] = audio_file_path
            return audio_file_path, False
        else:
            self.api_key_statuses[api_key] = False
            return None, False

    def process_file(self, filename):
        base_filename = os.path.basename(filename)
        temp_folder = f'tmp_output_{base_filename}'
        os.makedirs(temp_folder, exist_ok=True)

        subs = self.load_srt_file(filename)
        self.print_subtitles(subs, temp_folder)
        self.convert_audio_files(subs, temp_folder)
        self.combine_audio(subs, filename, temp_folder)

    def print_subtitles(self, subs, temp_folder):
        total_char_count = 0
        for index, sub in enumerate(subs, start=1):
            char_count = len(sub.text)
            for api_key in self.api_keys:
                if self.api_key_statuses[api_key]:
                    audio_file_path, from_cache = self.generate_audio(sub.text, index, api_key, temp_folder)
                    if audio_file_path:
                        new_audio_file_path = f'{temp_folder}/{index}.mp3'
                        if audio_file_path != new_audio_file_path:
                            shutil.copy(audio_file_path, new_audio_file_path)
                        if from_cache:
                            print(f"{index}. {sub.text} RC: 0")
                        else:
                            total_char_count += char_count
                            print(f"{index}. {sub.text} RC: {char_count}")
                        break
            else:
                print("Wszystkie klucze API są wyczerpane. Proszę dodać więcej kluczy.")

        print(f"Łączna liczba znaków: {total_char_count}")

    def load_srt_file(self, filename):
        return pysrt.open(filename, encoding='utf-8')

    def convert_audio_files(self, subs, temp_folder):
        for index, _ in enumerate(subs, start=1):
            audio_file = f'{temp_folder}/{index}.mp3'
            audio = AudioSegment.from_mp3(audio_file)
            audio.export(audio_file.replace('.mp3', '.wav'), format='wav')

    def combine_audio(self, subs, filename, temp_folder):
        combined = AudioSegment.silent(duration=0)

        for index, sub in enumerate(subs, start=1):
            start_time = sub.start.ordinal
            silence = AudioSegment.silent(duration=start_time - len(combined))
            combined += silence

            audio_file = f'{temp_folder}/{index}.wav'
            audio = AudioSegment.from_wav(audio_file)

            combined += audio

        output_filename = os.path.basename(filename).replace('.srt', '.eac3')
        combined.export(
            f"{self.output_folder}/{output_filename}", format='wav')
        os.system(
            f"ffmpeg -i {self.output_folder}/{output_filename} -c:a {self.output_folder}/{output_filename}")

    def process_all_files(self):
        for filename in glob.glob(f'{self.input_folder}/*.srt'):
            self.process_file(filename)

    def download_files(self):
        download_button = widgets.Button(
            description='Pobierz Pliki',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        download_button.on_click(self.download_files_callback)
        display(download_button)

    def download_files_callback(self, b):
        for filename in glob.glob(f'{self.output_folder}/*.eac3'):
            files.download(filename)

    def start_processing(self):
        cache_button = widgets.ToggleButton(
            value=self.use_cache,
            description='Użyj Cache - Zaoszczędź RC',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        cache_button.observe(self.update_cache_status, names='value')
        display(cache_button)

        start_button = widgets.Button(
            description='Wygeneruj Audio',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        start_button.on_click(self.start_processing_callback)
        display(start_button)

    def update_cache_status(self, change):
        self.use_cache = change['new']

    @execution_timer
    def start_processing_callback(self, b):
        self.process_all_files()
        self.download_files()

def main():
    display(HTML('<h1>ElevenLabs Infinite TTS API - Anime Lektor PL</h1>'))
    audio_generator = AudioGenerator()
    audio_generator.update_api_keys(api_keys)
    audio_generator.start_processing()

if __name__ == "__main__":
    main()

In [ ]:
# @title ElevenLabs Infinite TTS API - TEST asynchroniczności #1 { run: "auto" }
import os
import pysrt
import requests
import asyncio
from pydub import AudioSegment

# Wczytaj plik .srt i podziel go na części


def load_srt(file_path):
    return pysrt.open(file_path, encoding='utf-8')

# Wygeneruj asynchronicznie audio za pomocą API ElevenLabs
async def generate_audio(subtitle, api_key):
    url = "https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL"
    headers = {
        "Accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": api_key
    }
    data = {
        "text": subtitle.text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.75
        }
    }
    response = requests.post(url, json=data, headers=headers)
    with open(f'{subtitle.index}.wav', 'wb') as f:
        f.write(response.content)

# Połącz pliki audio w jeden duży plik audio
def merge_audio_files(subtitles):
    audio_files = [f"{subtitle.index}.wav" for subtitle in subtitles]
    combined_audio = AudioSegment.empty()
    for file in sorted(audio_files, key=lambda x: int(x.split('.')[0])):
        combined_audio += AudioSegment.from_wav(file)
    combined_audio.export("output.wav", format="wav")

# Główna funkcja
async def main():
    subtitles = load_srt('1.srt')
    api_keys = ['4d88c7168ddf8a163ffbf176276e72c5', '59f3ff34e070b562f8ac4f5847bf5f77']  # Lista kluczy API
    for subtitle in subtitles:
        for api_key in api_keys:
            await generate_audio(subtitle, api_key)
    merge_audio_files(subtitles)

# Uruchom główną funkcję
if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
# @title ElevenLabs Infinite TTS API - TEST asynchroniczności #2 - API 2 rządania na raz, i będzie ok { run: "auto" }
import requests
import asyncio
import aiohttp
import time
import nest_asyncio

nest_asyncio.apply()  # Zastosowanie nest_asyncio

CHUNK_SIZE = 1024
url = "https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL"

headers = {
  "Accept": "audio/mpeg",
  "Content-Type": "application/json",
  "xi-api-key": "8cea86b1109cd17fc38abdc5e773f19f"
}

# Lista rzeczy do powiedzenia
texts = ["Witaj świecie, jak się masz?",
         "To jest drugie zdanie.",
         "To jest trzecie zdanie."]

semaphore = asyncio.Semaphore(2)  # Utworzenie semafora z limitem 2

async def download_file(session, url, headers, data, file_name):
    async with semaphore, session.post(url, json=data, headers=headers) as response:
        if response.status != 200:
            raise Exception(f"Request to {url} returned status code {response.status}")
        with open(file_name, 'wb') as f:
            while True:
                chunk = await response.content.read(CHUNK_SIZE)
                if not chunk:
                    break
                f.write(chunk)

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i, text in enumerate(texts, start=1):
            data = {
                "text": text,
                "model_id": "eleven_multilingual_v2",
                "voice_settings": {
                    "stability": 0.5,
                    "similarity_boost": 0.75
                }
            }
            file_name = f'{i}.mp3'
            tasks.append(download_file(session, url, headers, data, file_name))
        await asyncio.gather(*tasks)

start_time = time.time()
await main()
end_time = time.time()
print("Zakończono wszystkie akcje. Czas wykonania: ", end_time - start_time, "sekund.")

In [ ]:
# @title ElevenLabs Infinite TTS API - TEST asynchroniczności #3 - Generowanie dla kilku kluczów api naraz błędy z api może mieć tylko 2 rządania na raz i pomoja tekst nwm jak to naprawić możliwość bycia szybkim, ale dużeo błędów do naprawienia { run: "auto" }

import pysrt
from pydub import AudioSegment
import os
import glob
import shutil
import nest_asyncio
import aiohttp
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import files
import asyncio
import random

# Inicjalizacja nest_asyncio
nest_asyncio.apply()

# Tworzenie klasy AudioGenerator
class AudioGenerator:
    def __init__(self, input_folder='input', output_folder='audio_output'):
        self.api_keys = []
        self.api_key_statuses = {key: True for key in self.api_keys}
        self.audio_cache = {}
        self.use_cache = False

        self.input_folder = input_folder
        self.output_folder = output_folder

        os.makedirs(input_folder, exist_ok=True)
        os.makedirs(output_folder, exist_ok=True)

    def update_api_keys(self, api_keys):
        self.api_keys = [key['key'] for key in api_keys]
        self.api_key_statuses = {key['key']: True for key in api_keys}

    async def generate_audio(self, text, index, temp_folder, char_count):
        CHUNK_SIZE = 1024
        random.shuffle(self.api_keys)

        for api_key in self.api_keys:
            if self.api_key_statuses[api_key]:
                url = "https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL" # Bella 0.5 0.75 | Stabilny Najlepszy
                headers = self.create_headers(api_key)
                data = self.create_data(text)

                audio_file_path = f'{temp_folder}/{index}.mp3'

                if self.use_cache and text in self.audio_cache:
                    return self.audio_cache[text], True, api_key

                async with aiohttp.ClientSession() as session:
                    async with session.post(url, json=data, headers=headers) as response:
                        if response.status == 200:
                            await self.write_audio_file(response, audio_file_path, CHUNK_SIZE)
                            self.audio_cache[text] = audio_file_path
                            return audio_file_path, False, api_key
                        else:
                            self.api_key_statuses[api_key] = False

        return None, False, None

    def create_headers(self, api_key):
        return {
            "Accept": "audio/mpeg",
            "Content-Type": "application/json",
            "xi-api-key": api_key
        }

    def create_data(self, text):
        return {
            "text": text,
            "model_id": "eleven_multilingual_v2",
            "voice_settings": {
                "stability": 0.5,
                "similarity_boost": 0.75
            }
        }

    async def write_audio_file(self, response, audio_file_path, CHUNK_SIZE):
        with open(audio_file_path, 'wb') as f:
            while True:
                chunk = await response.content.read(CHUNK_SIZE)
                if not chunk:
                    break
                f.write(chunk)

    async def process_file(self, filename):
        base_filename = os.path.basename(filename)
        temp_folder = self.create_temp_folder(base_filename)

        subs = self.load_srt_file(filename)
        await self.process_subtitles(subs, temp_folder)
        self.convert_audio_files(subs, temp_folder)
        self.combine_audio(subs, filename, temp_folder)

    def create_temp_folder(self, base_filename):
        temp_folder = f'tmp_output_{base_filename}'
        os.makedirs(temp_folder, exist_ok=True)
        return temp_folder

    async def process_subtitles(self, subs, temp_folder):
        total_char_count = 0
        for index, sub in enumerate(subs, start=1):
            char_count = len(sub.text)
            for api_key in self.api_keys:
                if self.api_key_statuses[api_key]:
                    audio_file_path, from_cache = await self.generate_audio(sub.text, index, api_key, temp_folder)
                    if audio_file_path:
                        new_audio_file_path = self.copy_audio_file(audio_file_path, temp_folder, index)
                        self.print_subtitle_info(index, sub.text, char_count, from_cache)
                        total_char_count += char_count if not from_cache else 0
                        break
            else:
                print("Wszystkie klucze API są wyczerpane. Proszę dodać więcej kluczy.")
        print(f"Łączna liczba znaków: {total_char_count}")

    async def process_subtitle(self, sub, index, temp_folder, char_count, semaphore):
        total_char_count = 0
        async with semaphore:
            audio_file_path, from_cache, api_key = await self.generate_audio(sub.text, index, temp_folder, char_count)
            if audio_file_path:
                new_audio_file_path = self.copy_audio_file(audio_file_path, temp_folder, index)
                self.print_subtitle_info(index, sub.text, char_count, from_cache, api_key)
                if not from_cache:
                    total_char_count += char_count
                return total_char_count

    def copy_audio_file(self, audio_file_path, temp_folder, index):
        new_audio_file_path = f'{temp_folder}/{index}.mp3'
        if audio_file_path != new_audio_file_path:
            shutil.copy(audio_file_path, new_audio_file_path)
        return new_audio_file_path

    def print_subtitle_info(self, index, text, char_count, from_cache, api_key):
        print(f"{index}. Napis: {text} | Liczba RC: {char_count if not from_cache else 0} | ID Klucza API: {api_key}")

    def load_srt_file(self, filename):
        return pysrt.open(filename, encoding='utf-8')

    def convert_audio_files(self, subs, temp_folder):
        for index, _ in enumerate(subs, start=1):
            audio_file = f'{temp_folder}/{index}.mp3'
            audio = AudioSegment.from_mp3(audio_file)
            audio.export(audio_file.replace('.mp3', '.wav'), format='wav')

    def combine_audio(self, subs, filename, temp_folder):
        combined = AudioSegment.silent(duration=0)

        for index, sub in enumerate(subs, start=1):
            start_time = sub.start.ordinal
            silence = AudioSegment.silent(duration=start_time - len(combined))
            combined += silence

            audio_file = f'{temp_folder}/{index}.wav'
            audio = AudioSegment.from_wav(audio_file)

            combined += audio

        output_filename = os.path.basename(filename).replace('.srt', '.eac3')
        combined.export(
            f"{self.output_folder}/{output_filename}", format='wav')
        os.system(
            f"ffmpeg -i {self.output_folder}/{output_filename} -c:a {self.output_folder}/{output_filename}")

    async def process_all_files(self):
        for filename in glob.glob(f'{self.input_folder}/*.srt'):
            await self.process_file(filename)

    def download_files(self):
        download_button = self.create_download_button()
        display(download_button)

    def create_download_button(self):
        download_button = widgets.Button(
            description='Pobierz Pliki',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        download_button.on_click(self.download_files_callback)
        return download_button

    def download_files_callback(self, b):
        for filename in glob.glob(f'{self.output_folder}/*.eac3'):
            files.download(filename)

    def start_processing(self):
        cache_button = self.create_cache_button()
        display(cache_button)

        start_button = self.create_start_button()
        display(start_button)

    def create_cache_button(self):
        cache_button = widgets.ToggleButton(
            value=self.use_cache,
            description='Użyj Cache - Zaoszczędź RC',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        cache_button.observe(self.update_cache_status, names='value')
        return cache_button

    def create_start_button(self):
        start_button = widgets.Button(
            description='Wygeneruj Audio',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        start_button.on_click(self.start_processing_callback)
        return start_button

    def update_cache_status(self, change):
        self.use_cache = change['new']

    def start_processing_callback(self, b):
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.process_all_files())
        self.download_files()


# Główna funkcja
def main():
    display(HTML('<h1>ElevenLabs Infinite TTS API - Anime Lektor PL</h1>'))
    audio_generator = AudioGenerator()
    audio_generator.update_api_keys(api_keys)
    audio_generator.start_processing()

if __name__ == "__main__":
    main()


In [ ]:
# @title ElevenLabs Infinite TTS API - Anime Lektor PL (asyncio) { run: "auto" }
import os
import shutil
import glob
import asyncio
import nest_asyncio
import aiohttp
import pysrt
from pydub import AudioSegment
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import files
from elevenlabs import voices as el_voices
from elevenlabs import set_api_key

#===============================================================================
# Button CSS
display(HTML('''
<style>
    .widget-button {
        font-weight: bold !important;
    }
    .widget-button.jupyter-widgets.widget-button.mod-active,
    .widget-button:active {
        background-color: #2d2e2e;
    }
    .widget-toggle-button {
        font-weight: bold !important;
    }
    .widget-toggle-button.jupyter-widgets.widget-toggle-button.mod-active,
    .widget-toggle-button:active {
        background-color: #2d2e2e;
    }
</style>
'''))
#===============================================================================

os.chdir('/content/')
nest_asyncio.apply()

class AudioGenerator:
    def __init__(self, input_folder='input', output_folder='audio_output'):
        self.api_keys = []
        self.api_key_statuses = {key: True for key in self.api_keys}
        self.audio_cache = {}
        self.use_cache = False

        self.input_folder = input_folder
        self.output_folder = output_folder

        os.makedirs(input_folder, exist_ok=True)
        os.makedirs(output_folder, exist_ok=True)

        self.voice_names = {
            'Brian': ['Brian', 'Brian - Slow speaking and deep'],
            'Bella': ['Bella'],
            'Scarlett': ['Scarlett'],
            'Knightley': ['Knightley'],
            'Joanne': ['Joanne'],
            'Nicole': ['Nicole']
        }

        self.voice_dropdown = widgets.Dropdown(options=list(self.voice_names.keys()))
        display(self.voice_dropdown)

    def update_api_keys(self, api_keys):
        self.api_keys = [key['key'] for key in api_keys]
        self.api_key_statuses = {key['key']: True for key in api_keys}

    def create_headers(self, api_key):
        return {
            "Accept": "audio/mpeg",
            "Content-Type": "application/json",
            "xi-api-key": api_key
        }

    def create_data(self, text):
        return {
            "text": text,
            "model_id": "eleven_multilingual_v2",
            "voice_settings": {
                "stability": 0.5,
                "similarity_boost": 0.75
            }
        }

    def find_voice_id(self, voice_list, voice_names):
        for voice in voice_list.voices:
            if voice.name in voice_names:
                return voice.voice_id
        return None

    async def generate_audio_url(self, api_key):
        set_api_key(api_key)
        voice_list = voices()
        voice_id = self.find_voice_id(voice_list, self.voice_names[self.voice_dropdown.value])
        if voice_id:
            return f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
        else:
            return None

    async def write_audio_file(self, response, audio_file_path, CHUNK_SIZE):
        with open(audio_file_path, 'wb') as f:
            while True:
                chunk = await response.content.read(CHUNK_SIZE)
                if not chunk:
                    break
                f.write(chunk)

    async def generate_audio(self, text, index, api_key, temp_folder):
        CHUNK_SIZE = 1024
        url = await self.generate_audio_url(api_key)
        if url is None:
            print(f"Nie znaleziono głosu dla klucza API: {api_key}")
            return None, False
        headers = self.create_headers(api_key)
        data = self.create_data(text)

        audio_file_path = f'{temp_folder}/{index}.mp3'

        if self.use_cache and text in self.audio_cache:
            return self.audio_cache[text], True

        async with aiohttp.ClientSession() as session:
            async with session.post(url, json=data, headers=headers) as response:
                if response.status == 200:
                    await self.write_audio_file(response, audio_file_path, CHUNK_SIZE)
                    self.audio_cache[text] = audio_file_path
                    return audio_file_path, False
                else:
                    self.api_key_statuses[api_key] = False
                    return None, False

    def copy_audio_file(self, audio_file_path, temp_folder, index):
        new_audio_file_path = f'{temp_folder}/{index}.mp3'
        if audio_file_path != new_audio_file_path:
            shutil.copy(audio_file_path, new_audio_file_path)
        return new_audio_file_path

    def print_subtitle_info(self, index, text, char_count, from_cache):
        print(f"{index}. {text} RC: {char_count if not from_cache else 0}")

    def load_srt_file(self, filename):
        return pysrt.open(filename, encoding='utf-8')

    def convert_audio_files(self, subs, temp_folder):
        for index, _ in enumerate(subs, start=1):
            audio_file = f'{temp_folder}/{index}.mp3'
            audio = AudioSegment.from_mp3(audio_file)
            audio.export(audio_file.replace('.mp3', '.wav'), format='wav')

    def combine_audio(self, subs, filename, temp_folder):
        combined = AudioSegment.silent(duration=0)

        for index, sub in enumerate(subs, start=1):
            start_time = sub.start.ordinal
            silence = AudioSegment.silent(duration=start_time - len(combined))
            combined += silence

            audio_file = f'{temp_folder}/{index}.wav'
            audio = AudioSegment.from_wav(audio_file)

            combined += audio

        output_filename = os.path.basename(filename).replace('.srt', '.eac3')
        combined.export(
            f"{self.output_folder}/{output_filename}", format='wav')
        os.system(
            f"ffmpeg -i {self.output_folder}/{output_filename} -c:a {self.output_folder}/{output_filename}")

    async def process_file(self, filename):
        print(f"Przetwarzanie pliku: {filename}")
        base_filename = os.path.basename(filename)
        temp_folder = self.create_temp_folder(base_filename)

        subs = self.load_srt_file(filename)
        await self.process_subtitles(subs, temp_folder)
        self.convert_audio_files(subs, temp_folder)
        self.combine_audio(subs, filename, temp_folder)
        print(f"Przetworzono plik: {filename}")

    def create_temp_folder(self, base_filename):
        temp_folder = f'tmp_output_{base_filename}'
        os.makedirs(temp_folder, exist_ok=True)
        return temp_folder

    async def process_subtitles(self, subs, temp_folder):
        total_char_count = 0
        for index, sub in enumerate(subs, start=1):
            char_count = len(sub.text)
            for api_key in self.api_keys:
                if self.api_key_statuses[api_key]:
                    audio_file_path, from_cache = await self.generate_audio(sub.text, index, api_key, temp_folder)
                    if audio_file_path:
                        new_audio_file_path = self.copy_audio_file(audio_file_path, temp_folder, index)
                        self.print_subtitle_info(index, sub.text, char_count, from_cache)
                        total_char_count += char_count if not from_cache else 0
                        break
            else:
                print("Wszystkie klucze API są wyczerpane. Proszę dodać więcej kluczy.")
        print(f"Łączna liczba znaków: {total_char_count}")

    async def process_all_files(self):
        for filename in glob.glob(f'{self.input_folder}/*.srt'):
            await self.process_file(filename)

    def download_files(self):
        download_button = self.create_download_button()
        display(download_button)

    def create_download_button(self):
        download_button = widgets.Button(
            description='Pobierz Pliki',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        download_button.on_click(self.download_files_callback)
        return download_button

    def download_files_callback(self, b):
        for filename in glob.glob(f'{self.output_folder}/*.eac3'):
            files.download(filename)

    def start_processing(self):
        cache_button = self.create_cache_button()
        display(cache_button)

        start_button = self.create_start_button()
        display(start_button)

    def create_cache_button(self):
        cache_button = widgets.ToggleButton(
            value=self.use_cache,
            description='Użyj Cache - Zaoszczędź RC',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        cache_button.observe(self.update_cache_status, names='value')
        return cache_button

    def create_start_button(self):
        start_button = widgets.Button(
            description='Wygeneruj Audio',
            style=dict(font_weight='bold'),
            layout=widgets.Layout(width='300px', height='30px')
        )
        start_button.on_click(self.start_processing_callback)
        return start_button

    def update_cache_status(self, change):
        self.use_cache = change['new']

    @execution_timer
    def start_processing_callback(self, b):
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.process_all_files())
        self.download_files()

def main():
    display(HTML('<h1>ElevenLabs Infinite TTS API - Anime Lektor PL</h1>'))
    audio_generator = AudioGenerator()
    audio_generator.update_api_keys(api_keys)
    audio_generator.start_processing()

if __name__ == "__main__":
    main()
